# Exporting the Breton example sentences

In [31]:
!pip install mwclient
!pip install mwparserfromhell
!pip install mwcomposerfromhell
!pip install jupyter
!pip install ipywidgets widgetsnbextension pandas-profiling


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [32]:
import datetime, tqdm, re, time, string

from mwclient import Site
#myparserfromhell : parser pour le code wiki
import mwparserfromhell, mwcomposerfromhell
import pickle

In [33]:
print('connecting...')
#site contenant tous les exemples
site = Site('arbres.iker.cnrs.fr/', path='/')
outf = open('fichiers_extraction/problemPages.html','w')
allpages = list(site.allpages())
print('got all', len(allpages), 'pages')

connecting...
got all 9613 pages


#### Charger toutes les pages du site: 

In [38]:
t = time.time()
#dictionnaire contenant toutes les pages aspirées
pages = {}
#on peut ici modifier le nombre de pages à charger pour charger petit à petit ex : allpages[:3000] pour les 3000 premières pages
# newlist=allpages
for p in tqdm.tqdm(allpages):
# for p in tqdm.tqdm(newlist):
    #si les pages ne sont pas de la documentation
    if not('/documentation' in p.name or '/docname' in p.name or 'Module:' in p.name):
        #on ajoute le nom de la page comme clé, son contenu comme valeur
        pages[p.name] = p.text()

print('extraction done. it took', time.time()-t, 'seconds.')

100%|██████████| 9613/9613 [19:16<00:00,  8.31it/s] 

extraction done. it took 1156.3399622440338 seconds.


### Compter tous les exemples trouvés dans les pages:

In [39]:
# Pour enregistrer les pages chargées dans un fichier Pickle: 
# une fois toutes les pages chargées, on le met dans un pickle
with open("fichiers_extraction/Pages.pickle", "wb") as myFile:
   pickle.dump(pages, myFile)
with open("fichiers_extraction/Pages.txt", "w") as myFile:
    for title,text in pages.items():
        myFile.write(title+'\n'+text+'\n------------------------\n\n')
    

In [40]:
# print(', '.join(pages.keys()))
#on créé un fichier txt contenant tous les titres de pages
open('fichiers_extraction/pageTitles.txt','w', encoding="utf-8").write('\n'.join(pages.keys()))
print('extracted',len(pages),'pages')


extracted 9613 pages


# Restart from local data

In [105]:
import datetime, tqdm, re, time, string
from mwclient import Site
import mwparserfromhell, mwcomposerfromhell
import pickle

with open('fichiers_extraction/Pages.pickle', 'rb') as f:
    pages = pickle.load(f)
open('fichiers_extraction/pageTitles.txt','w', encoding="utf-8").write('\n'.join(pages.keys()))
print('extracted',len(pages),'pages')

extracted 9613 pages


# wiki2lines

In [106]:
def wiki2lines(wikitext, title):
    """
    takes the wikitext of the example table and the title
    returns obj:
    obj = {'title':title,'phonetic':phonetic,'text':text, 'gloss':gloss, 'standardise':standardise,'lemgloss':lemgloss,'translation':translation, 'agrammaticale': agrammaticale, 'info':info}
    
    """
    contexte, phonetic, text, gloss, standardise, lemgloss, translation, agrammaticale, info = None, None, None, None, None, None, None, None, None
    rest = ''
    wikitext_split = wikitext.split('\n')
        
    # special case if the text is separated in two lines(line 1 and line 5), and the gloss is separated in line 3 and line 7, we put them together
    if "(4) ...|| gouarnamant || Pariz || ne || glask || nemed || eun || dra : || […]" in wikitext_split[1] or "bgcolor=#F0FF80|  gwenn," in wikitext_split[1] or "(4)|| Me || zo || bet || ouezan || ket || 'ta || marse … || marse || me || teir || gwezh" in wikitext_split[1] \
    or "(6) ... || gouarnamant || Pariz || ne || glask || nemed || eun dra: […]" in wikitext_split[1] or "(6)|| Ne || oa'''n''' || ket || gant || ma || c'hentañ || treizhadenn, || hogen || argadet || e voe || '''an douaradez''' || ac'han'''on'''." in wikitext_split[1] \
    or "(1)|| Pa || ne || oar || ket || '''an nen''', || pa || ne || re'''er''' || tamm || kegin || ebet," in wikitext_split[1] or "(2)|| Amañ || em eus || klasket || strollañ || ar pezh || a zo || bet || skrivet || gantañ || diwar-benn || ar politikerezh," in wikitext_split[1] \
    or "(4)||- Ur wech oa kouezhet klañv. An dra-se oa er bloavezh, a gav din, 1949." in wikitext_split[1] or "(1)|| Met || ne selle || ket-eñ || doc'h || ar re-se. " in wikitext_split[1] or "(1)|| Hag || an holl || o doa || pellaet || a-benn || doc'h || an oaled " in wikitext_split[1]\
    or "(7)|| Mar || deus || tud || hag a || zo || inouus, […], || ez eus || <u>lod all</u> " in wikitext_split[1] or "(3)|| <u>Ar re-mañ</u>, || evid || ar bloaz || kenta || e oant || e Huelgoad," in wikitext_split[1] or "(3)|| Me || zo || '''bet''' || || ouezan || ket || 'ta || marse … || '''marse''' || || '''me''' || teir || gwezh" in wikitext_split[1]\
    or "(2)|| Ha || n'oe || selaouet || anezhañ || da || wel || pelec'h || vie || bet || aretet." in wikitext_split[1] or "(1)|| Ret || eo || bout || graet || '''ar vicher-se''' || e-pad || pevar || blez || bennak || " in wikitext_split[1] or "(3)|| Lakaet || ez eus || bet || da || gefridi || dezhañ || ambroug || hag || aliañ || '''neb''' || den," in wikitext_split[1]\
    or "(4)|| '''Neb''' || n'en deus || gwelet, || keta, || e vez || roet || peoc'h || dezhañ" in wikitext_split[1] or "(7)|| '''Ret''' || eo || bout || graet || ar vicher-se || e-pad || pevar || blez || bennak ||" in wikitext_split[1]:
        wikitext_split[1] = wikitext_split[1]+wikitext_split[5]
        wikitext_split[3] = wikitext_split[3]+wikitext_split[7]
        wikitext_split[5]= '|-'
        wikitext_split[7]= '|-'
    # special case if the text is separated in two lines，and also the gloss and translation are separated in two lines, we put them together
    elif len(wikitext_split)> 12 and ("(3)|| E-lec'h || mont, || avat, || '''setu'''-hi || '''hag''' || azezañ || war ar || skabell-bediñ," in wikitext_split[1] or "(4)|| Ha koulskoude || he deus || ezhomm || a-walc'h || da gaout || un dra || bennak || da reiñ || d'he || bugale" in wikitext_split[1]):
        wikitext_split[1] = wikitext_split[1]+wikitext_split[5]
        wikitext_split[3] = wikitext_split[3]+wikitext_split[7]
        wikitext_split[9]= wikitext_split[9]+wikitext_split[11]
        wikitext_split[5]= '|-'
        wikitext_split[7]= '|-'
        wikitext_split[11]= '|-'
    # special case if the text is separated in three lines(line 1 and line 5 and line 9), and the gloss is separated in line 3 and line 7 and line 11, we put them together
    elif "chouk," in wikitext_split[1]:
        wikitext_split[1] = wikitext_split[1]+wikitext_split[5]+wikitext[9]
        wikitext_split[3] = wikitext_split[3]+wikitext_split[7]+wikitext[11]
        wikitext_split[5]= '|-'
        wikitext_split[7]= '|-'
        wikitext_split[9]= '|-'
        wikitext_split[11]= '|-'
    # special case if the third line includes "[[*]] || Kaset", we ignore it
    elif len(wikitext_split)> 4 and ("[[*]] || Kaset " in wikitext_split[3] or "[[*]] || MA || zi |||| HOR || yezh" in wikitext_split[3]):
        wikitext_split[3]= '|-'
    # special case if the gloss is seperated in two lines(line 3 and line 4), we put them together
    elif len(wikitext_split)> 4 and (" || [[POSS|leur]]<sup>[[2]]</sup> || [[lavar|langue]] || [[pemdeziek|quotidien]] || [[evel|comme]] || [[ma|que]] [[emañ|est]] || [[POSS|notre]] || [[hini|celui]]-nous" in wikitext_split[4] or "|| [[R]] [[zo|est]] || [[anavezout|conn]].[[-et (Adj.)|u]] || [[gant|par]].[[pronom incorporé|lui]]" in wikitext_split[4] or "|| [[forzh|force]] || \"costaud\"" in wikitext_split[4]):
        wikitext_split[3] = wikitext_split[3]+wikitext_split[4]
        wikitext_split[4]= '|-'
    # special case if the translation line not begain with "colspan=15", we add it in the beginning
    elif len(wikitext_split)> 8 and "||| 'Comme c'est stupide !' " in wikitext_split[7]:
        wikitext_split[7]= wikitext_split[7].replace("|||","||| colspan=\"15\" | ")
    elif len(wikitext_split)> 6 and "||| 'Je sais.'" in wikitext_split[5]:
        wikitext_split[5]= wikitext_split[5].replace("|||","||| colspan=\"15\" | ")
    elif len(wikitext_split)> 6 and "||| 'réincarner'" in wikitext_split[5]:
        wikitext_split[5]= wikitext_split[5].replace("|||","||| colspan=\"15\" | ")
    elif len(wikitext_split)> 6 and "'tel ou tel' || 'telle ou telle'" in wikitext_split[5]:
        wikitext_split[5]= wikitext_split[5].replace("|||","||| colspan=\"15\" | ")
    elif len(wikitext_split)> 6 and "||| Vous (ne) '''faites que''' prendre la mouche, aussi !'" in wikitext_split[5]:
        wikitext_split[5]= wikitext_split[5].replace("|||","||| colspan=\"15\" | ")
    elif len(wikitext_split)> 8 and "||| 'cent vingt, 120' || 'cent quarante, 140' || 'cent soixante, 160'" in wikitext_split[7]:
        wikitext_split[7]= wikitext_split[7].replace("|||","||| colspan=\"15\" | ")
    elif len(wikitext_split)> 8 and "||| 'autour de la table'" in wikitext_split[7]:
        wikitext_split[7]= wikitext_split[7].replace("|||","||| colspan=\"15\" | ")

    for i,line in enumerate(wikitext_split):
        if line.strip().startswith('|-') or 'prettytable' in line: 
            continue
        # if we find a line starting with "|(" and not a phonetic line, we have a contexte line
        if 'CONTEXTE' in line and not contexte:
            split_result = re.split(r'CONTEXTE', line)
            if len(split_result) > 1:
                contexte = split_result[1].strip()
                continue
        if line[:2]=='|(' and ("[…]" or "traou-ze," or "chouk,"not in line) and bool(re.search(r'</?font.*?>',line))==True and not phonetic:
            # if we find a <font> tag in the line, this is a phonetic line
            phonetic = line[1:]
        elif re.match(r'^\|', line) and 'CONTEXTE' not in line and not text:
            # otherwise first line will be text
            text = line[1:]
        
        elif ('Équivalent standardisé' in line or 'équivalent écriture standard' in line or 'équivalent standard' in line or 'interprétation' in line or "[[KLT]]" in line) and not standardise:
            # if the word "équivalent standardisé" is in the line, it's a "standardise" line
            standardise = line[1:]
        elif text and '[[' in line and not lemgloss and not 'colspan' in line and not "agrammatical" in line:
            # if there is “[[” in the line and not a colspan, then it's a lemgloss 
            # most of the time lemgloss is a "lemma-gloss"
            lemgloss = line[1:]
            continue
        elif text and re.match(r'^\|', line) and not lemgloss and not gloss:
            # if no [[ is found it's a gloss
            gloss = line[1:]
        elif 'colspan' in line and 'CONTEXTE' not in line and not translation:
            # first colspan should be the translation
            split_result = re.split(r'colspan="\d+".*?\|\s*', line)
            if len(split_result) > 1:
                translation = split_result[1].strip()
        elif 'colspan' in line and '| (' in line  and 'CONTEXTE' not in line:
            translation += '| ('+line.split('| (')[1]
        elif ('colspan' in line and '[[*]] ' in line or 'agrammatical' in line ) and not agrammaticale:
            # second colspan should be the agrammaticales
            split_result = re.split(r'\[\*\] ', line)
            if len(split_result) > 1:
                agrammaticale = split_result[1].strip()
        elif 'colspan' in line and not info:
            # second colspan should be the information about the source
            split_result = re.split(r'colspan="\d+".*?\|\s*', line)
            if len(split_result) > 1:
                info = split_result[1].strip()
        elif "colspan" in line and 'cité' in line:
            info += ' cité dans '+line.split('cité')[1]
        elif "colspan" in line and 'citée' in line:
            info += ' citée '+line.split('citée ')[1]
        elif "colspan" in line and '(noté' in line:
            info += '(noté' +line.split('(noté')[1]
        elif "colspan" in line and '[[' in line:
            info += '[[' +line.split('[[')[1]
        elif "colspan" in line and 'Anna' in line:
            info+= 'Anna' +line.split('Anna')[1]
        elif "colspan" in line and '[http:' in line:
            info+= '[http:' +line.split('[http:')[1]
        elif "colspan" in line and 'Finistère' in line:
            info+= 'Finistère' +line.split('Finistère')[1]
        elif "colspan" in line and 'comme' in line:
            info+= 'comme' +line.split('comme')[1]
        elif "colspan" in line and 'citant' in line:
            info+= 'citant' +line.split('citant')[1]
        elif "colspan" in line and 'interview par' in line:
            info+= 'interview par' +line.split('interview par')[1]
        #special case for "||||||||||||||| [[Plourin (2000)|Plourin (2000]]:28)"
        elif "|||||||||||||| " in line:
            info = '|||||||||||||| '+line.split('|||||||||||||| ')[1]
        elif len(line) > 2:
            rest+=line
    
    obj = {'title':title,'contexte': contexte, 'phonetic':phonetic,'text':text, 'gloss':gloss, 'standardise':standardise,'lemgloss':lemgloss,'translation':translation, 'agrammaticale': agrammaticale, 'info':info}
    obj = info2catsource(obj)

    
    if not obj['text'] or (not obj['gloss'] and not obj['lemgloss']) :
        rest += ' something missing!!! '
    if rest:
        return {**obj, 'rest':rest}

    return obj




def prettyprint(item):
    for key, value in item.items():
            print('____',key)
            print(str(value) )
    print( '__________________\n')



def info2catsource(obj):
    line_info = obj['info']
    if not line_info:
        return {**obj, 'category': None, 'source': None}
        
    category, source= None, None
    if line_info.startswith("''"):
        line_split = line_info.split("'', ") # split at '',_
        category = line_split[0][2:]
        if len(line_split) > 1:
            source = line_split[1]
    elif line_info.startswith("[["):
        # case xxxx
        source = line_info
    elif "', " in line_info:
        line_split = line_info.split("', ") # split at ,_
        category = line_split[0].replace("'","")
        source = line_split[1].replace("'","")

    else:
        source = line_info
        # print('Too many items: ', line_info)
        # qsdf
    if source:
        source = source.split('|')[-1].replace('[[','').replace(']]','')
        if re.search("(\'\'.*\'\' ?,?).*", source):                               
            other = re.search("(\'\'.*?\'\' ?,?).*", source).group(1)                                 
            source = source.replace(other, '').replace("''","") 
    # if ":" in source: 
    #     l = re.search("(.*):.*", source).group(1) + ')'
    # else: 
    # l = source
    # return object with categories and sources
    return {**obj, 'category':category, 'source':source} # **obj = spread obj


### Test pour la fonction avec une seule exemple

In [369]:

# obj = wiki2lines("""{| class="prettytable"
# |(1)|| Al || labous'''ed'''-mor […] || n'en em || vagont || ken || nemet || diwar || pesk'''ed'''.
# |-
# ||| [[art|le]] || [[labous|oiseaux]]-[[mor|mer]] || [[ne]]<sup>[[1]]</sup> [[en em|se]]<sup>[[1]]</sup> || [[magañ|nourrissent]] || [[ken|pas]] || [[nemet|seulement]] || [[diwar|de]] || [[pesk|poisson]].s  
# |-
# ||| colspan="15" | 'Les oiseaux de mer se nourrissent exclusivement de poisson.'
# |-
# ||||||||| colspan="15" | ''Cornouaillais (Bigouden)'', [[Bijer (2007)|Bijer (2007]]:134)
# |}""", 'test')
# prettyprint(obj)
# info2catsource(obj)

# Extraction globale

In [107]:
import re
import tqdm.notebook as tqdm

def write_errors(error_list, file_name):
    with open(file_name, 'w') as file_error:
        for item in error_list:
            for key, value in item.items():
                file_error.write(key + '\n')
                if key=="title":
                    file_error.write('https://arbres.iker.cnrs.fr/index.php?title='+str(value) + '\n') 
                else:
                    file_error.write(str(value) + '\n')
            file_error.write( '__________________\n')

    print(len(error_list),'errors in ', file_name)

exceptions = """
class="prettytable"  <take not>
class="prettytable" <take not>
|(6)|| bgcolor="#4682B4"|<i><font color=white>traduction</font></i> |
bgcolor="#4682B4"
''ur bern burbu'' || 'un tas de pustules' 
[[skuizh]].[[-entez]], n. f.
''kibri'''en'''''
kan'''er''' || 'chanteur' || vs. || kan'''iad''' ||
''[[kelenn (V.)|kelenn]]'''adur''''' || '(une) instruction'
(2)a. || ''c'hoant'''us''''' ||
''deread'''elezh''''' ||
|| ''dourek'' || 'qui contient de l'eau' ||
|| geri'''ennoù''' || 
|| [[nom abstrait]] d'action || traduction ||
|| ''kelennadur'' || '(une) instruction'
|| ''Divert'''issañ''''' 'divertir'
||<font color=green> [''''on''' ti] </font color=green> || eun ti || 'une maison'
|| Prend '''unan arall''' || ''3 ans 10 mois, 7 mois d'exposition à la langue''
|| Regarde '''heol 'zo bremañ'''. || ''4 ans 2 mois, 8 mois d'exposition à la langue''
|| ''deuit '''gant n'in'''.'' || 'venez avec moi' ||
|| ''ur GWIR vab'' || 'un vrai fils' ||<font color=green>[yr'gwi:rva:p]</font color=green> 
|| ''un hanTER torzh'' || 'une [[hanter|demi]] [[torzh|miche]]' ||<font color=green>
|| ''hanTER-kant'' || [[hanter|demi]]-cent', 50|| || ''Léonard (Cleder)''
|| ''ur gwir '''v'''ab || 'un vrai fils' || [[Kervella (1947)|Kervella (1947]]:§76)
|| ''e berr '''g'''omzoù'' || [[Kervella (1947)|Kervella (1947]]:§513)
|| ''ur brizh '''k'''risten'' || 'un mauvais chrétien' || [[Menard & Kadored (2001)|Menard & Kadored (2001]]:'brizh')
|| '''am'''zor 
<font color=green>1SG</font color=green>
(1)|| N'int || ket || aet || war-raok.
|| mot 1' mot 2 || mot 3 || mot 4 || mot 5-mot 6.
|| Pet || vloaz || en deus || ho || '''chas''' ? 
Ne || ra || ket || netra || vs. / || b.?? || Ne || ra || ket || NETRA
|| c'hoant'''us''' || '(homme) disposé à
|| cosas || que || '''no''' || sirven || || para || '''nada'''. |||| ''Castillan''
<i><font color=green>1SG</font></i>
|| ''[[kas]], '''de'''gas ||
équipe permanente porteuse de projet > ||
bgcolor=#5F9EA0
''[[glav]]'' || 'pluie' || '''''di'''c'hlav''
|||| Who do you think
bgcolor=#ccffcc
|| '''attestation avec /s-/''' ||
(A)|| assertions fortes, verbes déclaratifs || ''dire, rapporter, être vrai que, être certain que, être sur, être évident...
(3)|| ''X'' || lavar || ''un dra bennak'' || da ''unan bennak'' ||> || ''X'' [[lavarout|dire]] ''quelque chose'' à ''quelqu'un''
(p.9)|| ASTERIKS (dezhañ e unan): || '''Ez an d''' 'e heuliañ a-bell....
(1)|| colspan="15" |  אני מדברת ברטונית, ומה אתך?
(3)|| nom masculin || évidence |||| source
(3)|| ''GWALL vad'' || 'très bon' ||
(2)|| racine nominale ||> || reduplication
(2)|| 'qui ?' || '''''siapa''''' ||> 'quiconque' || ''siapa saja'', || ou '''''sapia-sapia'''''
(1)|| ''nom testé'' || ''accord'' || ''pronom'' || ''5  X'' || ''Niverus eo X||
<i><font color=green>3SGM</font></i> 
(1)|| déterminant || (adjectif) || nom || (adjectif) 
|| les [[articles]] || ''an, al, ar, un, ul, ur''
(1)|| Ar plant n' {int / [[*]] eo } ket glas. || (plantenn, || plantennoù)
Classe A:|| ''say, report, exclaim, assert, claim, vow, be true, be certain, be sure, be obvious
<u>Kapios fititis</u>
Magazennoù a oa, peogwir ar brikoli a oa gwerzhet de ar penn kwir, ne peze james da gont.'' 
<font color=green>[</font color=green><sub>[[AspP]]</sub> || Sujet  '''Asp°''' 
(1)|| || singulier || pluriel interne || pluriel suffixal || sources
* ar vugale || '''ne''' ||
carte [http:
(1)||<font color=green>/be/ || ''[[bezañ|beza]]'' || 'être'
|| syllabe || = || [<sub>syllabe</sub> || attaque || [<sub>rime</sub> [ || noyau ]  [ || coda ] || ] ]
(1)|| colspan="15" |Inventaire (non exhaustif) des attaques
Inventaire (non exhaustif) des codas
(1)|| ... |||| verbe || ... || <u><font color=red>sujet</font color=red></u> |||| <fo
(1) || ''Ober pas so hasou,'' || ''hep gaou pan dezraouet''
(3)|| '''krag'''-[[aotrou]], 'petit monsieur' || [[Merser (2009)]]
(1)|| Ha || c'hwi, || ma || flah || bihan, || ma || 'ho pije || gwelet || an traou-ze,
(1)|| nom féminin || évidence || traduction || source
'The boy raised his eyes.'
'I am inclined to say that the teacher is right.'
|(4)|| Sevel || a || reas || prim || he || brozh || du, ...
(24)|| a. ||*|| Musa || ba || nakàn || '''ni:''' || '''o'''. 
(5)|| ''Je suis allé'' || : || oeit '''on''' || / || '''em es''' oeit ||
(1)b. || '''''Deoc'h''' e ministraf a graf'' || 'Je vous 
|(3)a. || ''He dimiziff … '''ne''' riff '''quet''''' || 'Je ne l'épouserai pas.' 
(A)|| assertions fortes, [[verbes déclaratifs]] || '''''Lavaret''' en deus e vije deuet hirio''.
||| [[art|un]] agneau [[bihan|petit]] || [[debriñ|mang]].[[-et (Adj.)|é]] || par || [[art|un]] pourceau
Gris-'''en''' med lång svans grymtade. 
||| [[R]] [[kaout|avait]] || [[kaout|eu]] || [[c'hoant|envie]] || donner || [[da|à]].[[pronom incorporé|nous]] 
||| [[DEM|ceux.là]] || <font color=green>[</font color=green> [[evit|pour]] || [[art|le]] [[bloaz|an]] || [[les numéraux ordinaux|premier]] ||
''skañvder'', ''skañvded''
(1)|| ''rakkarvan'''egezh''''' || 'prognathisme' 
(2)|| 'A:ñi || ''' 'añ''' ||<font color=green>[</font color=green> S-kí'ikig || wui <font color=green>]</font color=green> || hí:.
(2)|| || singulier || pluriel interne || sources
(1)|| || singulier || pluriel interne || sources
(1)|| Setu, en ur zont d'ar gêr e choment a-sav eno neuze. 
(1)|| ''un doez'''en''''' || 'un épi' || ''en toez'''ad''''' || 'les épis d'un champ'
(3)|| '''''hur bout''' a ramb'' || 'nous avons'
(5)|| Elle (ne) '''fait que''' dormir de toute la journée ! 
(9)|| «Skrivet eo er brofeted: « '''Oll''' e vezint kelennet gand Doue». 
(5)|| c'hoarzh'''us''' > '''di'''c'hoarzh || pleg'''us''' > '''di'''bleg
(8)|| '''ki''' || > dour'''gi''', ''Standard'', [[Kervella (1947)|Kervella (1947]]:§299) || / || > '''ki'''dour, [[Favereau (1993)|Favereau (1993]]:'loutre')
<font color=green>1PL
(1)|| groupe nominal || > || reduplication
||| an den-mañ > || an den-mañ-''''n''' den || ar c'hi-mañ > 
|| raedenn || OK|| _ || _ || [[*]] || _ || _ || OK||
|| || 80 vl. || < 80 vl. || > 50 vl. || < 60 vl.
(1)|| bgcolor=#66CDAA| Flav! || bgcolor=#66CDAA| Klak ! || bgcolor=#66CDAA| Badadav! ||
(1)|| <font color=orange>La souris</font color=orange> || chatouille ||<font color=green>le chat</font color=green>. 
(2)|| ''nom testé'' || ''[[réciproques]]'' || ''[[quantifieurs flottants|Q flottants]] distr.'' || ''[[pep]] a X'' ||
(1)|| ''[[seurt]]'' 'sorte' || > || '''''lïes'''seurt'' 'multiple'
(1)|| Lavaret || o deus || || '''e veho''' || brao || an amzer || warc'hoazh.
|| maen || _ || _ || _ || _ || _ || _ || SG/ [[*]] PL|| _ || OK|| [[*]]
(9)|| { ? ur c'harredad / [[*]] ur bern } || gwez
(10)|| [[*]] ur c'hilo || gwez
(11)|| { [[*]] ul lodenn / [[*]] un tamm } || gwez
|| fubu || PL || PL || [[*]] || OK || OK || OK || SG/PL|| _ || _ || _
(4)|| '''ki''' || > dour'''gi''', ''Standard'', [[Kervella (1947)|Kervella (1947]]:§299) || / || > '''ki'''dour, [[Favereau (1993)|Favereau (1993]]:'loutre')
(9)|| «Skrivet eo er brofeted: « '''Oll''' e vezint kelennet gand Doue».
(1)|| ''[[-ed (N.)|-ed]]'' || ''elvo'''ed''''' 'peupleraie', [[Ménard (2012)]]
(3)|| Lavaret || o deus || || '''e veho''' || brao || an amzer || warc'hoazh.
(2)|| ''3SGM || 'M eus ket'''oñ''' || karet
(4)|| suffixe celtique insulaire > || nom verbal
(4)|| ''ar PaOur-kêz PaOtr'' || 'le pauvre garçon' ||<font color=green>[ar ˌpokɛz 'po:tr]</font color=green> || ''Plozévet'', [[Goyat (2012)|Goyat (2012]]:194)
(2)|| '''hanter'''-'''v'''ezw. ||
bgcolor=#FA8072
|| colspan="15" | <font color=orange>'''A'''</font color=orange> : 'Merci' / <font color=orange>'''B'''</font color=orange> : 'De rien.', 'C'est peu de chose.'
(3)|| <u>Ar re-mañ</u>, || evid || ar bloaz || kenta || e oant || e Huelgoad, 
""".strip().split('\n')


repretty = re.compile(r'\{\|.*?class="prettytable"((\n|.)*?)\|\}', re.MULTILINE)
count_exceptions = 0
alllineobjects = []
error_exemple = []
missing_translations = []
exampleFile = open('fichiers_extraction/examples.txt', 'w')
ignoredFile = open('fichiers_extraction/ignored.txt', 'w')
allc=0
for title in tqdm.tqdm(pages):
    exampleFile.write(title + '\n_____________________________\n')
    wikitext = pages[title]
    counter = 0

    for m in repretty.finditer(wikitext):
        # special tables that are not examples:
        # https://arbres.iker.cnrs.fr/index.php?title=-ach,_-aj&action=edit
        cont = False
        for ex in exceptions:
            if ex in m.group(0):
                count_exceptions += 1
                # Verifier les exemples qui ne sont pas pris en compte vient de quelles exceptions
                ignoredFile.write('https://arbres.iker.cnrs.fr/index.php?title='+title+'\n'+m.group(0)+'\n\n')
                cont = True
                break
        if cont: continue
        exampleFile.write('https://arbres.iker.cnrs.fr/index.php?title='+title+'\n')
        exampleFile.write(m.group(0) + '\n________________\n')
        result = wiki2lines(m.group(0), title)
        result['link'] = 'https://arbres.iker.cnrs.fr/index.php?title='+title
        alllineobjects.append(result)
        if result.get('rest'):
            error_exemple.append(result)
        if not result.get('translation') :
            missing_translations.append(result)
        # if  not result.get('rest') and not result.get('translation') :
        #     print('hello',result)
        counter += 1

    exampleFile.write('\n' + str(counter) + ' exemples trouvés dans la page ' + title + '\n================================\n\n\n')
    # Ajouter le nombre d'exemples trouvés à la variable totale
    allc+=counter
# Fermer le fichier
exampleFile.close()
# Afficher le nombre total d'exemples
print("Nous avons trouvés", allc,"exemples(y compris doublons)")

grouped_data = {}
for line_object in alllineobjects:
    translation = line_object['translation']
    if translation not in grouped_data:
        grouped_data[translation] = []
    grouped_data[translation].append(line_object)

nombre_example_list_plus_1 = 0
nombre_example_list_equal_1 = 0
with open('fichiers_extraction/exemples_répétés.txt', 'w') as file:
    for translation_key, example_list in grouped_data.items():
        if len(example_list) > 1:
            nombre_example_list_plus_1 += 1
            file.write('Il y a ' + str(len(example_list)) + " exemple(s) qui sont répétés dans ce groupe" + '\n')
            for example_obj in example_list:
                for k, v in example_obj.items():
                    file.write(k + '\n')
                    file.write(str(v) + '\n')
                file.write( '__________________\n')
            file.write( '__________________end___________________\n')
        else:
            nombre_example_list_equal_1 += 1
# Écrire les dictionnaires dans un fichier texte
with open('fichiers_extraction/all_line_objects.txt', 'w') as file:
    for item in alllineobjects:
        for key, value in item.items():
            file.write(key + '\n')
            file.write(str(value) + '\n')
        file.write( '__________________\n')

write_errors(error_exemple,'fichiers_extraction/error_exemple.txt')
write_errors(missing_translations,'fichiers_extraction/missing_translations.txt')
print(count_exceptions, "exemples in ignored.txt")
print(nombre_example_list_plus_1, "groupes d'exemples répétés dans all_line_objects.txt")
print(nombre_example_list_equal_1, "exemples uniques dans all_line_objects.txt")
print(nombre_example_list_plus_1+nombre_example_list_equal_1, "exemples au total sans doublons")

  0%|          | 0/9613 [00:00<?, ?it/s]

Nous avons trouvés 14893 exemples(y compris doublons)
0 errors in  fichiers_extraction/error_exemple.txt
0 errors in  fichiers_extraction/missing_translations.txt
346 exemples in ignored.txt
3716 groupes d'exemples répétés dans all_line_objects.txt
4515 exemples uniques dans all_line_objects.txt
8231 exemples au total sans doublons


# Create a couple.txt (store text,its translation, its categories, its source)

In [108]:
def cleanToken(t):
    t = t.replace("...", "")
    t = t.replace("tout.à.l'heure", "tout_à_l'heure")

    t = t.replace('<font color=green>', '')
    t = t.replace('</font color=green>', '')

    t = t.replace("<u>", "")
    t = t.replace("</u>", "")
    t = t.replace("<sub>", "")
    t = t.replace("</sub>", "")
    t = t.replace("<sub></sub>", "")
    return t



##################################################################################################
import re


def alignTokLemmgloss(text, lemgloss, phonetic):
    '''
    takes three lines of an example, produces triples
    '''
    pattern = r'\[\[(\w+)\|\|(\w+)\]\]'
    lemgloss = re.sub(pattern, r'[[\1|\2]]', lemgloss)

    text_list = text.split("||")
    lemgloss_list = lemgloss.split("||")
    phon_list = phonetic.split("||")
    couple_list = []

    if any("KLT" in tok or "Sujet" in tok for tok in text_list) or any(
            "KLT" in lem or "Sujet" in lem for lem in lemgloss_list):
        return couple_list

    text_list_stripped = [tok.strip() for tok in text_list]
    lemgloss_list_stripped = [lem.strip() for lem in lemgloss_list]

    text_new_list_stripped = []
    lemme_new_list_stripped = []

    for tok, lem in zip(text_list_stripped, lemgloss_list_stripped):
        if ']]-' in lem:
            parts = lem.split(']]-')
            if len(parts) > 2:
                lemme_new_list_stripped.append(parts[0] + ']]')
                lemme_new_list_stripped.append(parts[1] + ']]')
                lemme_new_list_stripped.append(parts[2])
            else:
                lemme_new_list_stripped.append(parts[0] + ']]')
                lemme_new_list_stripped.append('[[' + parts[1])

            word_to_find = parts[1].split('|')[0].replace('[[', '')

            if "-" + word_to_find in tok:
                text_parts = tok.split('-')
                if len(text_parts) > 2:
                    text_new_list_stripped.append(text_parts[0])
                    text_new_list_stripped.append("-" + text_parts[1])
                    text_new_list_stripped.append("-" + text_parts[2])
                else:
                    text_new_list_stripped.append(text_parts[0])
                    text_new_list_stripped.append("-" + text_parts[1])
            else:
                text_new_list_stripped.append(tok)
        elif any(word in tok.lower().split(" ") for word in ["un", "ul", "ur", "an", "al", "ar"]) and ' ' in tok:
        
            parts = tok.split(' ')
            new_tokens = []
            for i, part in enumerate(parts):
                if any(word in part.lower().split(" ") for word in ["un", "ul", "ur", "an", "al", "ar"]):
                
                    new_tokens.append(part)

                else:
                    new_tokens.append(part)

            text_new_list_stripped.extend(new_tokens)
            if any(mark in lem for mark in ["[[an, al, ar|le]]", "[[un, ul, ur|un]]"]):
        
                for mark in ["[[an, al, ar|le]]", "[[un, ul, ur|un]]"]:
                    if mark in lem:
                        index = lem.find(mark)
                        lem_parts = lem.split(mark)
                        lemme_new_list_stripped.extend([lem_parts[0] + mark, mark.join(lem_parts[1:])])
                        break
            else:
                lemme_new_list_stripped.append(lem)

        else:
            lemme_new_list_stripped.append(lem)
            text_new_list_stripped.append(tok)

    for index, lem in enumerate(lemme_new_list_stripped):
       ##traiter les exemples suivants:
       # <sup>[[1]]</sup>bouteill.[[-ad|ée]]"
       # <sup>[[1]]</sup>[[kêr|foyer]]
       #<sup>[[1]]</sup>[[paotr|homme]].[[-ed (PL.)|s]]
       # <sup>[[1]]</sup>[[moger|mur]].erie 
        pattern_mutchoice_right = r"<sup>\[\[(\d+)\]\]</sup>"
        if lem.startswith(" <sup>"):
            matches = re.findall(pattern_mutchoice_right, lem)
            if matches:
                index_left = -1
                for i in range(0, 4): 
                    index_left = lem.find('[', index_left + 1) 
                index_right = -1
                for i in range(0, 3): 
                    index_right = lem.find(']', index_right + 1)
                if index_left != -1 and index_right != -1:
                    lemme_new_list_stripped[index]= lem[:index_right] + 'Mut=' + matches[0] + 'Mutchoice=right' + lem[index_right:]
                    # print(lem)
    text_list_stripped = text_new_list_stripped
    lemgloss_list_stripped = lemme_new_list_stripped

    phon_list_stripped = [phon.strip() for phon in phon_list]

    if "standard" in text.lower() or "standard" in lemgloss.lower() or "Équivalent" in text or "Graphie" in text:
        return couple_list

    if text_list_stripped and text_list_stripped[-1] == "":
        text_list_stripped.pop()
    if lemgloss_list_stripped and lemgloss_list_stripped[-1] == "":
        lemgloss_list_stripped.pop()
    if lemgloss_list_stripped and lemgloss_list_stripped[-1] == "<elles>":
        lemgloss_list_stripped.pop()
    if lemgloss_list_stripped and lemgloss_list_stripped[-1] == "_":
        lemgloss_list_stripped.pop()
    if lemgloss_list_stripped and re.search(r'\d{4}', lemgloss_list_stripped[-1]):
        return couple_list
    if text_list_stripped and re.search(r'\d{4}', text_list_stripped[-1]):
        return couple_list
    if text_list_stripped and "colspan" in text_list_stripped[-1]:
        return couple_list
    if "[[*]]" in text_list_stripped:
        supprimer = text_list_stripped.index("[[*]]")
        text_list_stripped.pop(supprimer)
        # this indicates that the sentence is ungrammatical, and we skip it
        return couple_list

    if len(text_list_stripped) == len(lemgloss_list_stripped):

        if phonetic == "None":
            for tok, lemmgloss in zip(text_list_stripped, lemgloss_list_stripped):
                tok = cleanToken(tok)
                lemmgloss = cleanToken(lemmgloss)
                if tok and lemmgloss:
                    couple_list.append((tok, lemmgloss, 'None'))
        elif phonetic != 'None':
            for tok, lemmgloss, phone in zip(text_list_stripped, lemgloss_list_stripped, phon_list_stripped):
                tok = cleanToken(tok)
                lemmgloss = cleanToken(lemmgloss)
                phone = cleanToken(phone)
                if tok and lemmgloss and phone:
                    couple_list.append((tok, lemmgloss, phone))

    else:
        punctuation_marks = ['!', '.', ',', '?', ';', ':']
        for punctuation in punctuation_marks:
            if text_list_stripped and text_list_stripped[-1].endswith(punctuation) and not lemgloss_list_stripped[
                -1].endswith(punctuation):
                lemgloss_list_stripped.append(punctuation)

        if len(text_list_stripped) == len(lemgloss_list_stripped):
            if phonetic == "None":
                for tok, lemmgloss in zip(text_list_stripped, lemgloss_list_stripped):
                    tok = cleanToken(tok)
                    lemmgloss = cleanToken(lemmgloss)
                    if tok and lemmgloss:
                        couple_list.append((tok, lemmgloss, 'None'))
            elif phonetic != 'None':
                for tok, lemmgloss, phone in zip(text_list_stripped, lemgloss_list_stripped, phon_list_stripped):
                    tok = cleanToken(tok)
                    lemmgloss = cleanToken(lemmgloss)
                    phone = cleanToken(phone)
                    if tok and lemmgloss and phone:
                        couple_list.append((tok, lemmgloss, phone))
        else:
            for punctuation in punctuation_marks:
                if lemgloss_list_stripped and lemgloss_list_stripped[-1].endswith(punctuation) and not \
                        text_list_stripped[-1].endswith(punctuation):
                    text_list_stripped.append(punctuation)

            if len(text_list_stripped) == len(lemgloss_list_stripped):
                if phonetic == "None":
                    for tok, lemmgloss in zip(text_list_stripped, lemgloss_list_stripped):
                        tok = cleanToken(tok)
                        lemmgloss = cleanToken(lemmgloss)
                        if tok and lemmgloss:
                            couple_list.append((tok, lemmgloss, 'None'))
                elif phonetic != 'None':
                    for tok, lemmgloss, phone in zip(text_list_stripped, lemgloss_list_stripped, phon_list_stripped):
                        tok = cleanToken(tok)
                        lemmgloss = cleanToken(lemmgloss)
                        phone = cleanToken(phone)
                        if tok and lemmgloss and phone:
                            couple_list.append((tok, lemmgloss, phone))

            else:
                with open("fichiers_traitement/not_couple.txt", "a") as file:
                    file.write("text\n")
                    file.write(f"{text}\n")
                    file.write("lemgloss\n")
                    file.write(f"{lemgloss}\n")
    return couple_list


# make the couple into a dico including tok, lemma and gloss
def couple2tokobjects(triple):
    text, lem, phon = triple

    lem = re.sub(r"\[\[cardinal\|deux\]\]<sup>\[\[1\]\]</sup>", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinal\|deux\]\].F<sup>\[\[1\]\]</sup>", "[[div|deux]]", lem)
    lem = re.sub(r"\[\[cardinal\|quatre\]\]<sup>\[\[2\]\]</sup>", "[[peder|quatre]]", lem)

    lem = re.sub(r"<sup>\[\[2\]\]</sup> \[\[cardinal\|deux\]\]", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinal\|deux\]\]<sup>\[\[1\]\]</sup>", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinal\|2\]\]<sup>\[\[1\]\]</sup>", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinal\|cent\]\]", "[[gant|cent]]", lem)
    lem = re.sub(r"\[\[cardinal\|cent\]\]<sup>\[\[1\]\]</sup>", "[[kant|cent]]", lem)
    lem = re.sub(r"<sup>\[\[2\]\]</sup>\[\[cardinal\|cent\]\]", "[[c'hant|cent]]", lem)
    lem = re.sub(r"\[\[cardinal\|mille\]\]<sup>\[\[1\]\]</sup>", "[[mil|mille]]", lem)
    lem = re.sub(r"\[\[cardinal\|50\]\]<sup>\[\[1\]\]</sup>", "[[50|50]]", lem)
    lem = re.sub(r"\[\[cardinaux\|deux\]\]<sup>\[\[1\]\]</sup>", "[[zaou|deux]]", lem)
    lem = re.sub(r"\[\[dek\|dix\]\]<sup>\[\[1\]\]</sup>", "[[dek|dix]]", lem)
    lem = re.sub(r"\[\[cardinaux\|dix\]\]", "[[dek|dix]]", lem)
    lem = re.sub(r"\[\[cardinaux\|2\]\]<sup>\[\[1\]\]</sup>", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinaux\|deux\]\]", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinaux\|mille\]\]<sup>\[\[1\]\]</sup>", "[[mil|mille]]", lem)

    lem = re.sub(r"\[\[cardinal\|50\]\]", "[[50|50]]", lem)

    if 'diou' in text:
        lem = re.sub(r"\[\[cardinal\|deux\.F\]\]<sup>\[\[1\]\]</sup>", "[[diou|deux.F]]", lem)
    if 'vil' in text:
        lem = re.sub(r"<sup>\[\[1\]\]</sup>\[\[cardinal\|mille\]\]", "[[vil|mille]]", lem)
    if 'diouig' in text:
        lem = re.sub(r"\[\[cardinal\|2\]\]", "[[diou|2]]", lem)
    if "daou-ha-daou" in text:
        lem = re.sub(r"\[\[cardinal\|deux\]\]", "[[daou|deux]]", lem)

    lem = re.sub(r'<sup>(\[\[[^\[\]]+\]\])(.*?)</sup>',
                 lambda m: ''.join(f'<sup>{x}</sup>' for x in re.findall(r'\[\[[^\[\]]+\]\]', m.group(0))), lem)
    lem = re.sub('particule o', 'o', lem)

    lem = re.sub(r'<font color=green>.*?</font color=green>', '', lem)

    phon = re.sub(r'\[', '', phon)
    phon = re.sub(r'\]', '', phon)

    if '[[R]]' not in lem:
        match = re.search(r'<sup>\[\[(?!(1|4)\]\])(.*?)</sup>', lem)
        if match:
            k = match.group(2).strip(']')
            lem = re.sub(r'<sup>\[\[(?!(1|4)\]\]).*?</sup>', '[[R|_]]', lem)

        if "particule o" not in lem:
            lem = re.sub(r'<sup>\[\[4\]\]</sup>', '[[e|_]]', lem)

        lem = re.sub(r'<sup>\[\[1\]\]</sup>', '[[a|_]]', lem)

    if '[[R]]' in lem and '<sup>' in lem:
        match1 = re.search(r'\[\[R\]\]<sup>\[\[(?!(1|4)\]\])(.*?)</sup>', lem)
        if match1:
            k = match1.group(2).strip(']')
            lem = re.sub(r'\[\[R\]\]<sup>\[\[(?!(1|4)\]\])(.*?)</sup>', '[[R|_]]', lem)

        lem = re.sub(r'\[\[R\]\]<sup>\[\[4\]\]</sup>', '[[e|_]]', lem)
        lem = re.sub(r'\[\[R\]\]<sup>\[\[1\]\]</sup>', '[[a|_]]', lem)

    if '[[R]]' in lem and '<sup>' not in lem:
        lem = re.sub(r'\[\[R\]\]', '[[R|_]]', lem)
        k = 'R'

    if "'''" in text and '-' in lem and re.search(r'""".*?"""', text):
        extra_lem = re.search(r"'''(.*?)'''", text).group(1)
        match = re.search(r'(?<=\|{2}).*?-(.*?)(?=\|\|)', lem)

        extra_gloss = match.group(1).strip()
        lem = re.sub(r'-(.*?)\|\|', f'[[{extra_lem}|{extra_gloss}]] ||', lem)

    text_list = text.split('||')
    lem_list = lem.split('||')
    phone_list = phon.split('||')

    tok_objects = []

    for i in range(len(text_list)):
        tok = text_list[i].strip()
        lem_parts = lem_list[i].strip().split(']]')
        lem_sp = lem_list[i].strip().split('|')
        phonetic = phone_list[i].strip()

        import string

        tok_list = [part for part in tok.split(' ') if not all(char in string.punctuation for char in part)]
        phon_list = [part2 for part2 in phon.split(' ') if not all(char2 in string.punctuation for char2 in part2)]

        if len(lem_sp) == 1 and tok.startswith("'''"):
            tok_objects.append({
                'tok': tok.replace("'''", ''),
                'lemma': tok.replace("'''", ''),
                'gloss': lem_parts[0],
                'phonetic': phonetic.replace("'''", '')
            })


        elif len(lem_sp) == 1 and "<sub>" in lem_sp[0] and ('SC' or 'VP' or 'PredP' or 'CP' or 'DP') in lem_sp[0]:
            match = re.search(r'<sub>\[\[(.*?)\]\]', lem_sp[0])
            if match:
                content = match.group(1)
                tok_objects.append({
                    'PhraseStructure': content
                })

        elif len(lem_sp) == 1 and "<sub>" in lem_sp[0] and '[[' not in lem_sp[0]:
            match = re.search(r'<sub>(.*?)</sub>', lem_sp[0])
            if match:
                content = match.group(1)
                tok_objects.append({
                    'index': content
                })

        elif len(lem_sp) == 2 and '[[' in lem_sp[0]:

            gloss = lem_sp[1].rstrip('].')

            if 'cardinal' in lem_sp[0] and 'cardinal' not in tok:
                lem_sp[0] = tok
                tok_objects.append({
                    'tok': tok,
                    'lemma': tok,
                    'gloss': gloss,
                    'phonetic': phonetic if phonetic else '_'
                })
            if "cardinaux" in lem_sp[0]:
                lem_sp[0] = tok
                tok_objects.append({
                    'tok': tok,
                    'lemma': tok,
                    'gloss': gloss,
                    'phonetic': phonetic if phonetic else '_'
                })

            elif 'pronom in' in lem:
                lem_sp[0] = re.sub(r'\[\[pronom incorporé', 'me', lem_sp[0]) if 'moi' in gloss else ''
                lem_sp[0] = re.sub(r'\[\[pronom incorporé', 'te', lem_sp[0]) if 'toi' in gloss else ''
                lem_sp[0] = re.sub(r'\[\[pronom incorporé', 'eñ', lem_sp[0]) if 'lui' in gloss else ''
                lem_sp[0] = re.sub(r'\[\[pronom incorporé', 'hi', lem_sp[0]) if 'elle' in gloss else ''
                lem_sp[0] = re.sub(r'\[\[pronom incorporé', 'hon', lem_sp[0]) if 'nous' in gloss else ''
                lem_sp[0] = re.sub(r'\[\[pronom incorporé', 'hoc\'h-unan', lem_sp[0]) if 'vous' in gloss else ''
                lem_sp[0] = re.sub(r'\[\[pronom incorporé', 'int', lem_sp[0]) if 'eux' in gloss else ''



                tok_objects.append({
                    'tok': tok,
                    'lemma': lem_sp[0],
                    'gloss': gloss,
                    'phonetic': phonetic if phonetic else '_'
                })
            else:
                gloss = lem_sp[1].rstrip('].')
                tok_objects.append({
                    'tok': tok,
                    'lemma': lem_sp[0].strip('[]'),
                    'gloss': gloss,
                    'phonetic': phonetic if phonetic else '_'
                })


        elif len(tok_list) == len(lem_parts) - 1:
            for i in range(len(tok_list)):
                lem_inside = lem_parts[i].split('|')[0]
                gloss_inside = lem_parts[i].split('|')[-1]

                if 'numéraux' in lem_inside:
                    lem_inside = "[[" + tok

                if "cardinaux" in lem_inside:
                    lem_inside = "[[" + tok

                if '-ig' in lem_inside:
                    gloss_inside = 'petit'

                if 'cardinal' in lem_inside and 'cardinal' not in tok:
                    lem_inside = tok
                    pattern = r"[^\w\s]+"
                    lem_inside = re.sub(pattern, "", lem_inside)
                    lem_inside = '[[' + lem_inside

                if 'pronom incorporé' in lem_inside:
                    lem_inside = tok

                if len(tok_list) == len(phon_list):
                    tok_objects.append({'tok': tok_list[i],
                                        'lemma': lem_inside[2:],
                                        'gloss': gloss_inside,
                                        'phonetic': phon_list[i] if phon_list[i] else 'None'

                                        })
                else:
                    tok_objects.append({'tok': tok_list[i],
                                        'lemma': lem_inside[2:],
                                        'gloss': gloss_inside

                                        })
                    if phonetic != 'None':
                        with open('fichiers_traitement/error_phon.txt', 'a') as fi:
                            fi.write(f'text\n{text}\n')
                            fi.write(f'phonetic\n{phon}\n')
                            fi.write((f'______________\n'))




        else:
            tok_objects.append({
                'tok': tok,
                'lemma': '_',
                'gloss': '_',
                'phonetic': phonetic if phonetic else '_',
                'multiple': len(lem_parts) - 1})
            tok_obj = {'tok': tok, 'lemma': '_', 'gloss': '_', 'multiple': len(lem_parts) - 1}
            tok_n = 1

            for j in range(len(lem_parts)):
                if '[[' in lem_parts[j]:
                    lemma_gloss = lem_parts[j].strip('[[').split('|')
                    lemma = lemma_gloss[0].split('[[')[-1].strip()

                    if re.search(r'^-\w+', lemma):
                        if lemma != '-ig':
                            tok_objects.append({
                                'tok': lemma.strip(),
                                'lemma': lemma.strip(),
                                'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 and tok != '-ig' else ''
                            })

                        else:
                            tok_objects.append({
                                'tok': lemma.strip(),
                                'lemma': lemma.strip(),
                                'gloss': 'petit'
                            })




                    elif lemma_gloss == ['R', '_']:
                        if k != '+C':
                            tok_objects.append({
                                'tok': k,
                                'lemma': lemma.strip(),
                                'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''
                            })

                        else:
                            tok_objects.append({
                                'tok': k,
                                'lemma': lemma.strip(),
                                'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else '',
                                'Epenthesis': 'Yes'
                            })

                    elif lemma_gloss == ['a', '_']:
                        tok_objects.append({
                            'tok': '1',
                            'lemma': lemma.strip(),
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''
                        })

                    elif lemma_gloss == ['e', '_']:
                        tok_objects.append({
                            'tok': '4',
                            'lemma': lemma.strip(),
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''
                        })

                    elif 'cardinal' in lemma_gloss[0]:
                        tok_objects.append({
                            'tok': lemma_gloss[1],
                            'lemma': lemma_gloss[1],
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                    elif "cardinaux" in lemma_gloss[0] and 'cent' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'kant',
                            'lemma': 'kant',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )
                    elif 'cardi' in lemma_gloss[0] and '9' in lemma_gloss[1]:

                        tok_objects.append({
                            'tok': 'nav',
                            'lemma': 'nav',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                    elif "cardi" in lemma_gloss[0] and 'trois' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'tri',
                            'lemma': 'tri',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                    elif "cardi" in lemma_gloss[0] and 'vingt' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'ugent',
                            'lemma': 'ugent',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                    elif "cardi" in lemma_gloss[0] and '2' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'daou',
                            'lemma': 'daou',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                    elif "cardi" in lemma_gloss[0] and 'douze' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'daouzek',
                            'lemma': 'daouzek',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )
                    elif "cardi" in lemma_gloss[0] and 'deux' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'daou',
                            'lemma': 'daou',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )
                    elif "cardi" in lemma_gloss[0] and '3' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'tri',
                            'lemma': 'tri',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "cardi" in lemma_gloss[0] and 'un' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'unan',
                            'lemma': 'unan',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})

                    elif "pronom inco" in lemma_gloss[0] and 'lui' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'eñ',
                            'lemma': 'eñ',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'moi' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'me',
                            'lemma': 'me',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'toi' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'te',
                            'lemma': 'te',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'nous' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'hon',
                            'lemma': 'hon',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'elle' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'hi',
                            'lemma': 'hi',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'vous' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': "hoc'h-unan",
                            'lemma': "hoc'h-unan",
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'eux' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': "int",
                            'lemma': "int",
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'il' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': "enni",
                            'lemma': "enni",
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'ça' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': "se",
                            'lemma': "se",
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})


                    else:
                        tok_objects.append({
                            'tok': lemma.strip(),
                            'lemma': lemma.strip(),
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                        if '-ig' in tok_obj['tok'] and '-ig' in tok_obj['lemma']:
                            tok_obj['gloss'] = 'petit'

                        if 'numéraux' in tok_obj['lemma']:
                            tok_obj['lemma'] = tok_obj['tok']

                        if 'cardinal' in tok_obj['lemma'] and 'cardinal' not in tok_obj['tok']:
                            tok_obj['lemma'] = tok_obj['tok']
                            pattern = r"[^\w\s]+"
                            tok_obj['lemma'] = re.sub(pattern, "", tok_obj['lemma'])

                    tok_n += 1

    return tok_objects


#########################################
import re


def whole_lines():
    filename = "fichiers_extraction/all_line_objects.txt"
    all_examples = []

    with open(filename, "r") as file:
        lines = file.readlines()

    example = {}
    store_text = False
    store_lemgloss = False
    store_phonetic = False
    store_translation = False
    store_source = False
    store_category = False

    for line in lines:
        line = line.strip()

        if store_text:
            if line.startswith("||"):
                example["text"] = line[0:]
            else:
                example["text"] = line[3:]
            store_text = False
        elif store_lemgloss:
            example["lemgloss"] = line
            store_lemgloss = False
        elif store_phonetic:
            example["phonetic"] = line
            store_phonetic = False
        elif store_translation:
            example["translation"] = line
            store_translation = False
        elif store_category:
            example["category"] = line
            store_category = False
        elif store_source:
            example["source"] = line
            store_source = False
            if example["text"] != "" and example["lemgloss"] != "None":
                all_examples.append(example)
                example = {}  # Create a new dictionary for the next example

        if line == "text":
            store_text = True
        elif line == "lemgloss":
            store_lemgloss = True
        elif line == "phonetic":
            store_phonetic = True
        elif line == "translation":
            store_translation = True
        elif line == "category":
            store_category = True
        elif line == "source":
            store_source = True

    with open("fichiers_traitement/couple.txt", "w") as file:
        for example in all_examples:
            text = example["text"]
            lemgloss = example["lemgloss"]
            phonetic = example.get("phonetic", "None")
            translation = example.get("translation", "None")
            source = example.get("source", None)
            category = example.get("category", None)

            triplets = alignTokLemmgloss(text, lemgloss, phonetic)
            if triplets != []:
                file.write("text\n")
                file.write(cleanToken(text) + "\n")
                file.write("translation\n")
                file.write(cleanToken(translation)+ "\n")
                file.write("source\n")
                file.write(cleanToken(source)+ "\n")
                file.write("category\n")
                file.write(cleanToken(category)+ "\n")
    print("couple.txt created successfully!")


whole_lines()

couple.txt created successfully!


# Create triple.txt

In [109]:
import re
def couple2tokobjects(triple):
    text, lem, phon = triple

    lem = re.sub(r"\[\[cardinal\|deux\]\]<sup>\[\[1\]\]</sup>", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinal\|deux\]\].F<sup>\[\[1\]\]</sup>", "[[div|deux]]", lem)
    lem = re.sub(r"\[\[cardinal\|quatre\]\]<sup>\[\[2\]\]</sup>", "[[peder|quatre]]", lem)
    lem = re.sub(r"pronom incorporé","pronom_incorporé",lem)

    lem = re.sub(r"<sup>\[\[2\]\]</sup> \[\[cardinal\|deux\]\]", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinal\|deux\]\]<sup>\[\[1\]\]</sup>", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinal\|2\]\]<sup>\[\[1\]\]</sup>", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinal\|cent\]\]", "[[gant|cent]]", lem)
    lem = re.sub(r"\[\[cardinal\|cent\]\]<sup>\[\[1\]\]</sup>", "[[kant|cent]]", lem)
    lem = re.sub(r"<sup>\[\[2\]\]</sup>\[\[cardinal\|cent\]\]", "[[c'hant|cent]]", lem)
    lem = re.sub(r"\[\[cardinal\|mille\]\]<sup>\[\[1\]\]</sup>", "[[mil|mille]]", lem)
    lem = re.sub(r"\[\[cardinal\|50\]\]<sup>\[\[1\]\]</sup>", "[[50|50]]", lem)
    lem = re.sub(r"\[\[cardinaux\|deux\]\]<sup>\[\[1\]\]</sup>", "[[zaou|deux]]", lem)
    lem = re.sub(r"\[\[dek\|dix\]\]<sup>\[\[1\]\]</sup>", "[[dek|dix]]", lem)
    lem = re.sub(r"\[\[cardinaux\|dix\]\]", "[[dek|dix]]", lem)
    lem = re.sub(r"\[\[cardinaux\|2\]\]<sup>\[\[1\]\]</sup>", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinaux\|deux\]\]", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinaux\|mille\]\]<sup>\[\[1\]\]</sup>", "[[mil|mille]]", lem)
    lem = re.sub(r"\[\[DIM]]","[[-ig|petit]]",lem)
    lem = re.sub(r'\[\[(.*?)\]\]', lambda m: '[[' + re.sub(r'\s+', '', m.group(1)) + ']]', lem)
    lem = re.sub(r"\[\[tout]]","[[tout|tout]]",lem)
    lem = re.sub(r"\[\[R\]\]<sup>\[\[\+C\]\],\[\[4\]\]</sup>", "[[R]]<sup>[[4]]</sup>",lem)
    lem = re.sub(r"\[\[R\]\]<sup>\[\[4\]\] /(\+C)?</sup>",'[[R]]<sup>[[4]]</sup>',lem)
    lem = re.sub(r"\[\[R\]\]<sup>\[\[4\]\] / \[\[\+C\]\]</sup>",'[[R]]<sup>[[4]]</sup>',lem)

    lem = re.sub(r"\[\[ne\|ne\]\]<sup>\[\[1\]\]</sup>\[R\]", "[[ne|ne]][[a|_]]",lem)
    lem = re.sub(r"\[\[ne\|ne\]\]<sup>\[\[4\]\]</sup>\[R\]", "[[ne|ne]][[e|_]]",lem)
    lem = re.sub(r"\[\[ne\|ne\]\]<sup>\[\[R\]\]</sup>", "[[ne|ne]][[a|_]]",lem)
    lem = re.sub(r"\[\[ne\|ne\]\]<sup>\[\[1\]\]</sup>","[[ne|ne]]",lem)
    lem = re.sub(r"\[\[ne\|ne\]\]<sup>\[\[4\]\]</sup>","[[ne|ne]]",lem)

    lem=re.sub(r"\[\[pronom_incorporé\|moi]]","[[me|moi]]",lem)
    lem=re.sub(r"\[\[pronom_incorporé\|toi]]","[[te|toi]]",lem)
    lem=re.sub(r"\[\[pronom_incorporé\|lui]]","[[eñ|lui]]",lem)
    lem=re.sub(r"\[\[pronom_incorporé\|elle]]","[[he|elle]]",lem)
    lem=re.sub(r"\[\[pronom_incorporé\|nous]]","[[ni|nous]]",lem)
    lem=re.sub(r"\[\[pronom_incorporé\|vous]]","[[c’hwi|vous]]",lem)
    lem=re.sub(r"\[\[pronom_incorporé\|eux]]","[[int|eux]]",lem)
    lem=re.sub(r"\[\[pronom_incorporé\|elles]]","[[int|elles]]",lem)

    lem=re.sub(r"1SG",'[[-m|moi]]',lem)
    lem=re.sub(r"2SG","[[-c'h|toi]]",lem)
    lem=re.sub(r"3SGM","[[en|lui]]",lem)
    lem=re.sub(r"3SGF","[[he|elle]]",lem)
    lem=re.sub(r"1PL","[[hon|nous]]",lem)
    lem=re.sub(r"2PL","[[hoc'h|vous]]",lem)
    lem=re.sub(r"3PL","[[o|eux,elles]]",lem)

    lem=re.sub(r"2\.\[\[kaout", "[[kaout2", lem)
    lem=re.sub(r"1\.\[\[kaout", "[[kaout1", lem)
    lem = re.sub(r"3\.\[\[kaout", "[[kaout3", lem)

    text=re.sub(r"am-eus",'am eus',text)
    text=re.sub(r"ac'h-eus","ac'h eus",text)
    text=re.sub(r"en-deus",'en deus',text)
    text=re.sub(r'he-deus','he deus',text)
    text=re.sub(r'hon-deus','hon deus',text)
    text=re.sub(r'ho-peus','ho peus',text)
    text=re.sub(r'o-deus','o deus',text)
    text = re.sub(r"\('n\)", "'n", text)

    lem = re.sub(r"\[\[cardinal\|50\]\]", "[[50|50]]", lem)

    if 'diou' in text:
        lem = re.sub(r"\[\[cardinal\|deux\.F\]\]<sup>\[\[1\]\]</sup>", "[[diou|deux.F]]", lem)
    if 'vil' in text:
        lem = re.sub(r"<sup>\[\[1\]\]</sup>\[\[cardinal\|mille\]\]", "[[vil|mille]]", lem)
    if 'diouig' in text:
        lem = re.sub(r"\[\[cardinal\|2\]\]", "[[diou|2]]", lem)
    if "daou-ha-daou" in text:
        lem = re.sub(r"\[\[cardinal\|deux\]\]", "[[daou|deux]]", lem)

    lem = re.sub(r'<sup>(\[\[[^\[\]]+\]\])(.*?)</sup>',
                 lambda m: ''.join(f'<sup>{x}</sup>' for x in re.findall(r'\[\[[^\[\]]+\]\]', m.group(0))), lem)
    lem = re.sub('particule o', 'o', lem)

    lem = re.sub(r'<font color=green>.*?</font color=green>', '', lem)

    phon = re.sub('\\[', '', phon)
    phon = re.sub(']', '', phon)

    if '[[R]]' not in lem:
        lem = re.sub(r'<sup>.*?</sup>', '', lem)

        if "particule o" not in lem:
            lem = re.sub(r'<sup>\[\[4\]\]</sup>', '[[e|_]]', lem)

        lem = re.sub(r'<sup>\[\[1\]\]</sup>', '[[a|_]]', lem)

    if '[[R]]' in lem and '<sup>' in lem:
        match1 = re.search(r'\[\[R\]\]<sup>\[\[(?!(1|4)\]\])(.*?)</sup>', lem)
        if match1:
            k = match1.group(2).strip(']')
            lem = re.sub(r'\[\[R\]\]<sup>\[\[(?!(1|4)\]\])(.*?)</sup>', '[[R|_]]', lem)

        lem = re.sub(r'\[\[R\]\]<sup>\[\[4\]\]</sup>', '[[e|_]]', lem)
        lem = re.sub(r'\[\[R\]\]<sup>\[\[1\]\]</sup>', '[[a|_]]', lem)

    if '[[R]]' in lem and '<sup>' not in lem:
        lem = re.sub(r'\[\[R\]\]', '[[R|_]]', lem)
        k = 'R'

    text_list = text.split('||')
    lem_list = lem.split('||')
    phone_list = phon.split('||')

    tok_objects = []



    for i in range(len(text_list)):

        tok = text_list[i].strip()
        lem_parts = lem_list[i].strip().split(']]')
        lem_sp = lem_list[i].strip().split('|')
        phonetic = phone_list[i].strip()

        new_lem=lem_list[i].strip()

        if not (text_list[i].startswith("'''") and text_list[i].endswith("'''")):

            ex_lem= re.findall(r"'''(.*?)'''", text_list[i]) if re.findall(r"'''(.*?)'''", text_list[i]) else ""
            ex_gloss_r = re.findall(r'(.*?)\.\]\]', lem_list[i][::-1]) if re.findall(r'(.*?)\.\]\]',
                                                                                     lem_list[i][::-1]) else ''
            ex_gloss = ex_gloss_r[0][::-1] if ex_gloss_r else ''

            if ex_lem and ex_gloss and '|' not in ex_gloss:

                text_list[i] = re.sub("'''", ' ', text_list[i])
                text_list[i] = re.sub("'''", ' ', text_list[i])
                escaped_ex_gloss = re.escape(ex_gloss)
                lem_list[i] = re.sub(f'{escaped_ex_gloss}$', f"[[{ex_lem[0]}|{escaped_ex_gloss}]]", lem_list[i])
                tok = text_list[i].strip()
                lem_parts = lem_list[i].strip().split(']]')
                lem_sp = lem_list[i].strip().split('|')
                phonetic = phone_list[i].strip()

                new_lem = lem_list[i].strip()



        import string

        tok_list = [part for part in tok.split(' ') ]
        phon_list = [part2 for part2 in phon.split(' ') if not all(char2 in string.punctuation for char2 in part2)]
        lemma_list= [part3 for part3 in new_lem.split(' ') ]


        if len(lem_sp) == 1 and tok.startswith("'''") :
            tok_objects.append({
                'tok': tok.replace("'''", ''),
                'lemma': tok.replace("'''", ''),
                'gloss': lem_parts[0],
                'phonetic': phonetic.replace("'''", '')
            })


        elif len(lem_sp) == 1 and "<sub>" in lem_sp[0] and ('SC' or 'VP' or 'PredP' or 'CP' or 'DP') in lem_sp[0]:
            match = re.search(r'<sub>\[\[(.*?)\]\]', lem_sp[0])
            gloss_unique=lem_sp[0].split()[0]
            if match:
                content = match.group(1)
                tok_objects.append({
                    'tok':tok,
                    'lemma':tok,
                    'gloss':gloss_unique,
                    'PhraseStructure': content
                })


        elif len(lem_sp) == 1 and "<sub>" in lem_sp[0] and '[[' not in lem_sp[0]:
            match = re.search(r'<sub>(.*?)</sub>', lem_sp[0])
            if match:
                content = match.group(1)
                tok_objects.append({
                    'tok':tok,
                    'lemma':tok,
                    'gloss':lem_sp[0].split()[0],
                    'index': content.strip("'")
                })

        elif len(lem_sp) == 2 and '[[' in lem_sp[0]:

            gloss = lem_sp[1]
            if not re.match(r'\b\w+\b', gloss):
                gloss = gloss[:-1]

            if 'cardinal' in lem_sp[0] and 'cardinal' not in tok:
                lem_sp[0] = tok
                tok_objects.append({
                    'tok': tok,
                    'lemma': tok,
                    'gloss': gloss,
                    'phonetic': phonetic if phonetic else '_'
                })
            if "cardinaux" in lem_sp[0]:
                lem_sp[0] = tok
                tok_objects.append({
                    'tok': tok,
                    'lemma': tok,
                    'gloss': gloss,
                    'phonetic': phonetic if phonetic else '_'
                })



                tok_objects.append({
                    'tok': tok,
                    'lemma': lem_sp[0],
                    'gloss': gloss,
                    'phonetic': phonetic if phonetic else '_'
                })
            else:
                gloss = lem_sp[1].rstrip('].')
                tok_objects.append({
                    'tok': tok,
                    'lemma': lem_sp[0].strip('[]'),
                    'gloss': gloss,
                    'phonetic': phonetic if phonetic else '_'
                })


        elif len(tok_list) == len(lemma_list) and len(tok_list)>1:

            for i in range(len(tok_list)):
                if "]]-[[" not in lemma_list[i] and lemma_list[i].count("]]")==1:

                    lem_inside=lemma_list[i].split('|')[0]
                    gloss_inside=lemma_list[i].split('|')[-1]
                    if len(tok_list) == len(phon_list):
                        tok_objects.append({'tok': tok_list[i],
                                            'lemma': lem_inside[2:],
                                            'gloss': gloss_inside.strip(']'),
                                            'phonetic': phon_list[i] if phon_list[i] else 'None'

                                            })
                    else:
                        tok_objects.append({'tok': tok_list[i],
                                            'lemma': lem_inside[2:],
                                            'gloss': gloss_inside.rstrip(']')

                                            })
                elif "]]-[[" in lemma_list[i] and '-' in tok_list[i]:
                    comp_list2 = lemma_list[i].split(']]-[[')
                    tok_objects.append({
                        'tok': tok_list[i].rstrip(']'),
                        'lemma': '_',
                        'gloss': '_',
                        'phonetic': phonetic if phonetic else '_',
                        'multiple': len(lemma_list[i].split(']]-[['))})

                    for i in range(len(comp_list2)):
                        if '|' in comp_list2[i]:
                            parts2 = comp_list2[i].split('|')
                            tok_objects.append({
                                'tok': parts2[0].lstrip('['),
                                'lemma': parts2[0].lstrip('['),
                                'gloss': parts2[1].rstrip(']'),
                                'phonetic': phonetic if phonetic else '_'})

                elif "]].[[" in lemma_list[i]:

                    comp_list=lemma_list[i].split(']].[[')
                    tok_objects.append({
                        'tok': tok_list[i].rstrip(']'),
                        'lemma': '_',
                        'gloss': '_',
                        'phonetic': phonetic if phonetic else '_',
                        'multiple': len(lemma_list[i].split(']].[['))})

                    for i in range(len(comp_list)):
                        if '|' in comp_list[i]:
                            parts = comp_list[i].split('|')
                            tok_objects.append({
                                'tok': parts[0].lstrip('['),
                                'lemma': parts[0].lstrip('['),
                                'gloss': parts[1].rstrip(']'),
                                'phonetic': phonetic if phonetic else '_'})


        else:
            if len(lem_parts) > 1:

                tok_objects.append({
                    'tok': tok,
                    'lemma': '_',
                    'gloss': '_',
                    'phonetic': phonetic if phonetic else '_',
                    'multiple': len(lem_parts) - 1})
                tok_obj = {'tok': tok, 'lemma': '_', 'gloss': '_', 'multiple': len(lem_parts) - 1}
                tok_n = 1
            else:
                tok_objects.append({
                    'tok': tok,
                    'lemma': '_',
                    'gloss': '_',
                    'phonetic': phonetic if phonetic else '_'})
                tok_obj = {'tok': tok, 'lemma': '_', 'gloss': '_'}
                tok_n = 1

            for j in range(len(lem_parts)):
                if '[[' in lem_parts[j]:
                    lemma_gloss = lem_parts[j].strip('[[').split('|')
                    lemma = lemma_gloss[0].split('[[')[-1].strip()

                    if re.search(r'^-\w+', lemma):
                        if lemma != '-ig':
                            tok_objects.append({
                                'tok': lemma.strip(),
                                'lemma': lemma.strip(),
                                'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 and tok != '-ig' else ''
                            })

                        else:
                            tok_objects.append({
                                'tok': lemma.strip(),
                                'lemma': lemma.strip(),
                                'gloss': 'petit'
                            })

                    elif lemma_gloss == ['R', '_']:
                        if k != '+C':
                            tok_objects.append({
                                'tok': k,
                                'lemma': lemma.strip(),
                                'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''
                            })

                        else:
                            tok_objects.append({
                                'tok': k,
                                'lemma': lemma.strip(),
                                'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else '',
                                'Epenthesis': 'Yes'
                            })

                    elif lemma_gloss == ['a', '_']:
                        tok_objects.append({
                            'tok': 'a',
                            'lemma': 'R',
                            'gloss': '_'
                        })

                    elif lemma_gloss == ['e', '_']:
                        tok_objects.append({
                            'tok': 'e',
                            'lemma': 'R',
                            'gloss': '_'
                        })

                    elif 'cardinal' in lemma_gloss[0]:
                        tok_objects.append({
                            'tok': lemma_gloss[1],
                            'lemma': lemma_gloss[1],
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                    elif "cardinaux" in lemma_gloss[0] and 'cent' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'kant',
                            'lemma': 'kant',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )
                    elif 'cardi' in lemma_gloss[0] and '9' in lemma_gloss[1]:

                        tok_objects.append({
                            'tok': 'nav',
                            'lemma': 'nav',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                    elif "cardi" in lemma_gloss[0] and 'trois' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'tri',
                            'lemma': 'tri',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                    elif "cardi" in lemma_gloss[0] and 'vingt' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'ugent',
                            'lemma': 'ugent',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                    elif "cardi" in lemma_gloss[0] and '2' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'daou',
                            'lemma': 'daou',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                    elif "cardi" in lemma_gloss[0] and 'douze' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'daouzek',
                            'lemma': 'daouzek',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )
                    elif "cardi" in lemma_gloss[0] and 'deux' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'daou',
                            'lemma': 'daou',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )
                    elif "cardi" in lemma_gloss[0] and '3' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'tri',
                            'lemma': 'tri',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "cardi" in lemma_gloss[0] and 'un' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'unan',
                            'lemma': 'unan',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})

                    elif "pronom inco" in lemma_gloss[0] and 'lui' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'eñ',
                            'lemma': 'eñ',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'moi' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'me',
                            'lemma': 'me',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'toi' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'te',
                            'lemma': 'te',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'nous' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'hon',
                            'lemma': 'hon',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'elle' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'hi',
                            'lemma': 'hi',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'vous' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': "hoc'h-unan",
                            'lemma': "hoc'h-unan",
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'eux' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': "int",
                            'lemma': "int",
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'il' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': "enni",
                            'lemma': "enni",
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'ça' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': "se",
                            'lemma': "se",
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})


                    else:
                        tok_objects.append({
                            'tok': lemma.strip(),
                            'lemma': lemma.strip(),
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                        if '-ig' in tok_obj['tok'] and '-ig' in tok_obj['lemma']:
                            tok_obj['gloss'] = 'petit'

                        if 'numéraux' in tok_obj['lemma']:
                            tok_obj['lemma'] = tok_obj['tok']

                        if 'cardinal' in tok_obj['lemma'] and 'cardinal' not in tok_obj['tok']:
                            tok_obj['lemma'] = tok_obj['tok']
                            pattern = r"[^\w\s]+"
                            tok_obj['lemma'] = re.sub(pattern, "", tok_obj['lemma'])

                    tok_n += 1

    return tok_objects


# Store the output in triple.txt (attention : clean the existed triple.txt file before running the following code)

In [110]:
import re


def whole_lines():
    filename = "fichiers_extraction/all_line_objects.txt"
    all_examples = []

    with open(filename, "r") as file:
        lines = file.readlines()

    example = {}
    store_text = False
    store_lemgloss = False
    store_phonetic = False

    for line in lines:
        line = line.strip()

        if store_text:
            if line.startswith("||"):
                example["text"] = line[0:]
            else:
                example["text"] = line[3:]
            store_text = False
        elif store_lemgloss:
            example["lemgloss"] = line
            store_lemgloss = False

            if example["text"] != "" and example["lemgloss"] != "None":
                all_examples.append(example)

            example = {}
        elif store_phonetic:
            example["phonetic"] = line
            store_phonetic = False

        if line == "text":
            store_text = True
        elif line == "lemgloss":
            store_lemgloss = True
        elif line == "phonetic":
            store_phonetic = True

    # Process each example
    for example in all_examples:
        text = example["text"]

        lemgloss = example["lemgloss"]


        phonetic = example.get("phonetic", None)

        triplets = alignTokLemmgloss(text, lemgloss,phonetic)
        tok_objects_list = []


        for triple in triplets:
            tok_objects = couple2tokobjects(triple)
            tok_objects_list.extend(tok_objects)

        if tok_objects_list:
            with open('fichiers_traitement/triple.txt', 'a') as file:
                print(tok_objects_list,file=file)


whole_lines()

# Create n_triple.txt for punctuation

In [111]:
import re
def process_triple_file():
    with open("fichiers_traitement/triple.txt", "r") as file:
        lines = file.readlines()

    new_lines = []
    for line in lines:
        dicos = eval(line.strip())

        new_dicos = []
        pending_punctuation = None
        pending_punctuation_count = 0
        for dico in dicos:
            tok = dico.get("tok", "")
            gloss=dico.get('gloss','')
            lemma=dico.get('lemma','')
            lemma=re.sub('^<','',lemma)

            # Check if there is a pending punctuation to be added
            if pending_punctuation is not None:
                if pending_punctuation_count > 0:
                    pending_punctuation_count -= 1
                if pending_punctuation_count == 0:
                    new_dicos.append(pending_punctuation)
                    pending_punctuation = None           
            if tok.endswith((".", ",", "!", "?", ";", ":", "…")):
                if 'multiple' not in dico.keys():
                # Create a new dico with tok value without punctuation
                    new_dico = {
                        "tok": re.sub(r'\([^)]*\)', '', tok).rstrip(".!,?;:><…"),
                        "lemma": lemma.rstrip(".!,?;:><…"),
                        "gloss": gloss.rstrip("<>…"),
                        "phonetic": "None"
                    }
                    if new_dico["tok"]: 
                        new_dicos.append(new_dico)

                    # Create a dico for the punctuation
                    punctuation_dico = {
                        "tok": tok[-1],
                        "lemma": tok[-1],
                        "gloss": tok[-1],
                        "phonetic": "None"
                    }
                    new_dicos.append(punctuation_dico)
                else:
                    new_dico = {
                        "tok": re.sub(r'\([^)]*\)', '', tok).rstrip(".!,?;:><…"),
                        "lemma": lemma.rstrip(".!,?;:><…"),
                        "gloss": gloss.rstrip("<>…"),
                        "phonetic": "None",
                        "multiple": dico["multiple"]
                    }
                    if new_dico["tok"]:
                        new_dicos.append(new_dico)

                    # Create a dico for the punctuation
                    punctuation_dico = {
                        "tok": tok[-1],
                        "lemma": tok[-1],
                        "gloss": tok[-1],
                        "phonetic": "None"
                    }
                    pending_punctuation = punctuation_dico
                    pending_punctuation_count = dico["multiple"]+ 1
            else:
                new_dicos.append(dico)



        if pending_punctuation is not None:
            new_dicos.append(pending_punctuation)
        new_line = str(new_dicos) + "\n"
        new_lines.append(new_line)

    with open("fichiers_traitement/n_triple.txt", "w") as file:
        file.writelines(new_lines)

    print("n_triple.txt created successfully!")


process_triple_file()

n_triple.txt created successfully!


# Get the pos 

In [112]:
relinks = re.compile(r"\[\[(.*?)\]\]")

def get_wikicode(title):
    wikicode=''
    # on traite les redirections vers d'autres pages
    if '#REDIRECTION' in pages.get(title, ''):             
        #on suit la redirection
        newtitle = relinks.search(pages[title]).group(1)                
        newtitle = newtitle[0].upper()+newtitle[1:]
        if '_' in newtitle: 
            newtitle = newtitle.replace("_", " ")            
        #et on regarde à nouveau si le titre est dans le dictionnaire pages
        if newtitle in pages:
            wikicode = pages[newtitle]              
        else: 
            newtitle = newtitle[0].upper()+newtitle[1:].replace(' ','_')
            if newtitle in pages:
                wikicode = pages[newtitle]
            else:
                newtitle = newtitle.split(',')[0]
                newtitle = newtitle[0].upper()+newtitle[1:]
                if newtitle in pages:
                    wikicode = pages[newtitle]
                else:
                    wikicode='__currentPage:\nstrange redirect to page that does not exist: '+newtitle
                    print(wikicode+'\n')

    else:
        wikicode = pages.get(title, '')  
    return wikicode

In [113]:
rePOS = re.compile(
    r'verbes?|auxiliaires?|copules?|adverbes?|complémenteurs?|conjonctions?|prépositions?|adjectifs?|noms?|particules verbales?|interjections?|postpositions?|déterminants?|quantifieurs?|pronoms?|noms? propres?|suffixe|interrogatifs?|préfixes?|modaux|pluriels?|indéfinis?|particules? de discours|finales?|exclamatifs?',
    re.IGNORECASE)

lefffl = [li.strip() for li in open('fichiers_lefff/lefff-2.1.txt').read().split('\n') if li.strip() and li[0] != '#']
lefff = {li.split('\t')[0]: li.split('\t')[2] for li in lefffl}


# print(lefff)

import re

def add_pos(tok_obj):
    if 'pos' in tok_obj and tok_obj['pos']:
        return tok_obj  # pos already there

    # Check if the tok is a punctuation mark
    if 'tok' in tok_obj and re.match(r'^[\.,!?:;…]$', tok_obj['tok']):
        tok_obj['pos'] = 'PUNCT'
        return tok_obj

    if 'tok' in tok_obj and tok_obj['tok']=='R':
        tok_obj['pos']='SCONJ'
        return tok_obj
    tok=tok_obj.get('tok','')

    if tok == '-m':
        tok_obj['lemma'] = 'me'
        tok_obj['pos'] = 'PRON'
        return tok_obj
    if tok == "-c'h":
        tok_obj['lemma'] = 'te'
        tok_obj['pos'] = 'PRON'
        return tok_obj
    if tok == 'en':
        tok_obj['lemma']= 'eñ'
        tok_obj['pos'] = 'PRON'
        return tok_obj
    if tok == 'he':
        tok_obj['lemma'] = 'he'
        tok_obj['pos'] = 'PRON'
        return tok_obj
    if tok == 'hon':
        tok_obj['lemma']= 'ni'
        tok_obj['pos'] = 'PRON'
        return tok_obj
    if tok == "hoc'h":
        tok_obj['lemma']= "c'hwi"
        tok_obj['pos'] = 'PRON'
        return tok_obj
    if tok == 'o':
        tok_obj['lemma']= 'int'
        tok_obj['pos'] = 'PRON'
        return tok_obj

    lemma=tok_obj.get('lemma','')
    if lemma.startswith('-'):
        tok_obj['pos']='suffixe'
        return tok_obj

    if lemma.endswith('-'):
        tok_obj['pos']='prefixe'
        return tok_obj

    if 'lemma' in tok_obj:
        m = rePOS.search(tok_obj['lemma'])
        if m:
            tok_obj['pos'] = m.group(0)
            return tok_obj

    # is the pos in the gloss?
    if 'gloss' in tok_obj:
        m = rePOS.search(tok_obj['gloss'])
        if m:
            tok_obj['pos'] = m.group(0)
            return tok_obj

    # let's check the page for the lemma
    wikicode = get_wikicode(str.capitalize(tok_obj.get('lemma', '')))  # TODO: improve as this might fail sometimes
    # print(wikicode[:222])
    m = rePOS.search(wikicode)
    if m:
        tok_obj['pos'] = m.group(0)
        return tok_obj


    # let's use the gloss to make an educated guess about the pos using lefff
    if tok_obj.get('gloss', '') in lefff:
        tok_obj['pos'] = lefff[tok_obj.get('gloss', '')]

    return tok_obj

# Store the output in nt.txt

In [114]:
def write_processed_output(file_path, output_file_path):
    import ast
    with open(file_path, 'r') as file:
        lines = file.readlines()

    with open(output_file_path, "w") as output_file:
        for line in lines:
            tok_objects = ast.literal_eval(line)
            modified_tok_objects = [add_pos(tok_obj) for tok_obj in tok_objects]

            # Write the processed tok_obj objects to the output file line by line
            output_file.write(str(modified_tok_objects) + "\n")




write_processed_output('fichiers_traitement/n_triple.txt','fichiers_traitement/nt.txt')


# Preparation for the conllu

In [115]:
import re

def generate_conll_output(tok_objects, text, text_fr, source, sent_id, category):
    conll_output = ""
    conll_id = 1
    inside_id = 1

    for tok_obj in tok_objects:
        if isinstance(tok_obj, dict) and len(tok_obj) == 1:
            field_key, field_value = next(iter(tok_obj.items()))
            conll_line = f"{conll_id}\t{field_key}\t{field_value}\t_\t_\t_\t_\t_\t_\t_\n"
        else:


            tok_obj['tok'] = re.sub(r'\([^()]*\)', '', tok_obj['tok']).rstrip(".!,?;:") if re.search(r'\([^()]*\)',tok_obj['tok']) else tok_obj['tok']

            if 'multiple' not in tok_obj:
                if 'pos' in tok_obj and 'phonetic' in tok_obj:
                    if tok_obj['phonetic'] != 'None':
                        conll_line = f"{conll_id}\t{tok_obj['tok']}\t{tok_obj['lemma']}\t{tok_obj['pos']}\t_\t{tok_obj['phonetic']}\t_\t_\t_\tGloss={tok_obj['gloss']}\n"
                    else:
                        conll_line = f"{conll_id}\t{tok_obj['tok']}\t{tok_obj['lemma']}\t{tok_obj['pos']}\t_\t_\t_\t_\t_\tGloss={tok_obj['gloss']}\n"

                    conll_id += 1
                    inside_id = conll_id
                else:
                    if 'pos' in tok_obj:
                        conll_line = f"{conll_id}\t{tok_obj['tok']}\t{tok_obj['lemma']}\t{tok_obj['pos']}\t_\t_\t_\t_\t_\tGloss={tok_obj['gloss']}\n"
                        conll_id += 1
                        inside_id = conll_id
                    else:
                        conll_line = f"{conll_id}\t{tok_obj['tok']}\t{tok_obj['lemma']}\t_\t_\t_\t_\t_\t_\tGloss={tok_obj['gloss']}\n"
                        conll_id += 1
                        inside_id = conll_id
            elif 'multiple' in tok_obj:
                if tok_obj['phonetic'] != 'None':
                    conll_line = f"{inside_id}-{inside_id + tok_obj['multiple'] - 1}\t{tok_obj['tok']}\t{tok_obj['lemma']}\t_\t{tok_obj['phonetic']}\t_\t_\t_\t_\t_\n"
                else:
                    conll_line = f"{inside_id}-{inside_id + tok_obj['multiple'] - 1}\t{tok_obj['tok']}\t{tok_obj['lemma']}\t_\t_\t_\t_\t_\t_\t_\n"
                # # print all the mots composés
                # print("sent_id:", sent_id)
                # print("text:", text)
                # print("text_fr:", text_fr)
                # print("conll_line:", conll_line)

                
            conll_line = re.sub(']]', ' ', conll_line)
            conll_line = re.sub('\\[\\[kaout', 'kaout', conll_line)
            conll_line = re.sub("'''", '', conll_line)
            conll_line = re.sub('\\[', '', conll_line)

            lemma=tok_obj.get('lemma','')
            gloss=tok_obj.get('gloss','')
            tok=tok_obj.get('tok','')
            pos=tok_obj.get('pos','_')

            if tok == '-m' and lemma =='me':
                conll_line = re.sub(f"-m\tme\tPRON\t_",f"-m\tme\tPRON\tNumber=Sing|Person=1|PronType=Prs",conll_line)
            if tok == "-c'h" and lemma =='te':
                conll_line = re.sub(f"-c'h\tte\tPRON\t_",f"-c'h\tte\tPRON\tNumber=Sing|Person=2|PronType=Prs",conll_line)
            if tok == 'en' and lemma =='eñ':
                conll_line = re.sub(f"en\teñ\tPRON\t_",f"en\teñ\tPRON\tGender=Masc|Number=Sing|Person=3|PronType=Prs",conll_line)
            if tok == 'he' and lemma =='he':
                conll_line = re.sub(f"he\the\tPRON\t_",f"he\the\tPRON\tGender=Fem|Number=Sing|Person=3|PronType=Prs",conll_line)
            if tok == 'hon' and lemma =='ni':
                conll_line = re.sub(f"hon\tni\tPRON\t_",f"hon\tni\tPRON\tNumber=Plur|Person=1|PronType=Prs",conll_line)
            if tok == "hoc'h" and lemma =="c'hwi":
                conll_line = re.sub(f"hoc'h\tc'hwi\tPRON\t_",f"hoc'h\tc'hwi\tPRON\tNumber=Plur|Person=2|PronType=Prs",conll_line)
            if tok == "o" and lemma =="int":
                conll_line = re.sub(f"o\tint\tPRON\t_",f"o\tint\tPRON\tNumber=Plur|Person=3|PronType=Prs",conll_line)

            if tok_obj.get('PhraseStructure','')!='':
                conll_line=re.sub(f"{lemma}\t{pos}\t_",f"{lemma}\t{pos}\tPhraseStructure={tok_obj['PhraseStructure']}",conll_line)
            if tok_obj.get('index','')!='':
                conll_line=re.sub(f"{lemma}\t{pos}\t_",f"{lemma}\t{pos}\tindex={tok_obj['index']}",conll_line)
            if (lemma=='a' and gloss=='_') or (tok=='a' and lemma=='R'):
                conll_line=re.sub(f"{lemma}\t{pos}\t_",f"{lemma}\t{pos}\tMut=1",conll_line)
            if (lemma=='e' and gloss=='_') or (tok=='e' and lemma=='R'):
                conll_line=re.sub(f"{lemma}\t{pos}\t_",f"{lemma}\t{pos}\tMut=4",conll_line)
            
            if lemma=='kaout1':
                conll_line=re.sub(f"{tok}\t{lemma}\t{pos}\t_",f'kaout\tkaout\t{pos}\tPerson=1',conll_line)
            if lemma=='kaout2':
                conll_line=re.sub(f"{tok}\t{lemma}\t{pos}\t_",f'kaout\tkaout\t{pos}\tPerson=2',conll_line)
            if lemma=='kaout3':
                conll_line=re.sub(f"{tok}\t{lemma}\t{pos}\t_",f'kaout\tkaout\t{pos}\tPerson=3',conll_line)


        conll_output += conll_line.strip() + "\n"

    # Check if the last element of the text line is a punctuation mark
    text_elements = text.strip().split()
    if text_elements:  # Check if the list is not empty
        last_text_element = text_elements[-1][-1]
    # last_text_element = text.strip().split()[-1][-1]
        if last_text_element in (".", ",", "!", "?", ";", ":","…"):
            # Check if the last line of the conll output has this punctuation
            last_line_tokens = conll_output.strip().split('\n')[-1]

            if 'PUNCT' not in last_line_tokens:
                # Add the punctuation token to the conll output
                conll_id += 1
                conll_line = f"{conll_id}\t{last_text_element}\t{last_text_element}\tPUNCT\t_\t_\t_\t_\t_\tgloss={last_text_element}\n"
                conll_output += conll_line.strip() + "\n"
    if "("in category:
        parts = category.split(' (')
        if len(parts) > 1:
            dialect = parts[0]
            location_parts = parts[1].split(')')
            if location_parts:
                location = location_parts[0]
            else:
                location = 'None'
                pass
        else:
            dialect = category
            location = 'None'
            pass
    else:
        dialect = category
        location = 'None'
    conll_output = f"# sent_id = {source.replace(' ', '') +'__'+str(sent_id)}\n" + \
                f"# text = {text.replace('||', '')}\n" + \
                f"# text_fr = {text_fr}\n" + \
                f"# dialect = {dialect}\n" + \
                f"# location = {location}\n" + \
                f"# source = {source}\n" + \
                f"# status = WIP\n" + \
                conll_output + "\n"
    

    return conll_output



# Store the output in the conllu file

In [116]:
import ast
import re

def process_tokens_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    conll_output = ""
    sent_id = 1  # Initialize sent_id to 1

    with open("fichiers_traitement/couple.txt", "r") as couple_file:
        couple_lines = couple_file.readlines()
        couple_lines = [line.strip() for line in couple_lines]

    for i, line in enumerate(lines):
        if i * 8 + 1 >= len(couple_lines) or i * 8 + 3 >= len(couple_lines) or i * 8 + 5 >= len(couple_lines) or i * 8 + 7 >= len(couple_lines):
            # Skip this iteration if indices are out of range
            continue

        tok_objects = ast.literal_eval(line)
        text = couple_lines[i * 8 + 1].replace("text", "").strip()
        text_fr = couple_lines[i * 8 + 3].replace("translation", "").strip()
        text = re.sub("'''", '', text)
        source = couple_lines[i * 8 + 5].replace("source", "").strip()
        category = couple_lines[i * 8 + 7].replace("category", "").strip()
        # Assuming generate_conll_output is a function defined elsewhere
        conll_output += generate_conll_output(tok_objects, text, text_fr, source, sent_id,category)
        sent_id += 1  # Increment sent_id for the next example

    return conll_output

# Assuming 'nt.txt' and 'generate_conll_output' are correctly set up
output = process_tokens_file('fichiers_traitement/nt.txt')
with open('fichiers_traitement/none2_original.conllu', 'w') as file:
    print(output, file=file)


# Ajouter des modifications sur le fichier none2_original.txt généré

In [117]:
import re

def remove_trailing_punctuation(word):
    # Expression régulière pour supprimer la ponctuation en fin de mot
    return re.sub(r'[^\w\s-]+$', '', word)

def apply_replacement_rules(parts):  
    # Appliquer différentes règles de remplacement en fonction de la position de la colonne et de la valeur cible
    if len(parts) > 3:
        # Règle 1 : Si la troisième colonne est "an,al,ar", alors remplacez la quatrième colonne par "défini"
        if parts[5] != "_":
            parts[5]= "_"
        elif parts[9] == "Gloss=leur":
            parts[4] = "Number=Plur|Person=3|PronType=Prs"
        elif parts[9] == "Gloss=son" and parts[4]=="_":
            parts[4] = "Number=Sing|Person=3|PronType=Prs"
        elif parts[9] == "Gloss=sa" and parts[4]== "_":
            parts[4] = "Number=Sing|Person=3|PronType=Prs"
        elif parts[9] == "Gloss=ton" and parts[4]=="_":
            parts[4] = "Number=Sing|Person=2|PronType=Prs"
        elif parts[9] == "Gloss=elle" and parts[4]=="_":
            parts[4]= "Gender=Fem|Number=Sing|Person=3|PronType=Prs"
        elif parts[9] == "Gloss=lui" and parts[4]=="_":
            parts[4]= "Gender=Masc|Number=Sing|Person=3|PronType=Prs"
            if parts[1] == "eñ" and parts[2] == "eñ":
                parts[1] = "añ"
                parts[3] = "pronom"
        elif parts[9] == "Gloss=il" and parts[4]=="_":
            parts[4]= "Gender=Masc|Number=Sing|Person=3|PronType=Prs"
        elif parts[9] == "Gloss=nous" and parts[4]=="_":
            parts[4]= "Number=Plur|Person=1|PronType=Prs"
        elif parts[9] == "Gloss=je" and parts[4]=="_":
            parts[4]= "Number=Sing|Person=1|PronType=Prs"
        elif parts[9] == "Gloss=vous" and parts[4]=="_":
            parts[4]= "Number=Plur|Person=2|PronType=Prs"
        elif parts[9] == "Gloss=tu" and parts[4]=="_":
            parts[4]= "Number=Sing|Person=2|PronType=Prs"
        elif parts[9]=="Gloss=me" and parts[4]=="_":
            parts[4]="Number=Sing|Person=1|PronType=Prs"
        elif parts[9]=="Gloss=te" and parts[4]=="_":
            parts[4]="Number=Sing|Person=2|PronType=Prs"
        elif parts[9]=="Gloss=toi" and parts[4]=="_":
            parts[4]="Number=Sing|Person=2|PronType=Prs"
        elif parts[9]=="Gloss=moi" and parts[4]=="_":
            parts[4]="Number=Sing|Person=1|PronType=Prs"
        elif parts[9]=="Gloss=elles" and parts[4]=="_":
            parts[4]= "Gender=Fem|Number=Plur|Person=3|PronType=Prs"
        elif parts[9]=="Gloss=ils" and parts[4]=="_":
            parts[4]= "Gender=Masc|Number=Plur|Person=3|PronType=Prs"
        elif parts[2] == "an,al,ar":
            parts[3] = "défini"
        elif parts[2] == "un,ul,ur":
            parts[3] = "indéfini"
        # Règles 4-7 : Remplacer la quatrième colonne en fonction de certaines valeurs de la troisième colonne
        elif parts[2] in ["POP", "COP", "zo", "eo"]:
            replacement = "pronom" if parts[2] == "POP" else "AUX"
            parts[3] = replacement

        # Règle 8 : Remplacement basé sur des conditions impliquant plusieurs colonnes spécifiques
        elif len(parts) > 4 and parts[2] == "e" and parts[3] == "copule":
            parts[3] = "SCONJ"

        # Règle 9 : Si la deuxième colonne est "a" et la troisième colonne est "R", alors remplacez la quatrième colonne par "SCONJ"
        elif parts[1] == "a" and parts[2] == "R":
            parts[3] = "SCONJ"
        elif parts[9] == "Gloss=er" and parts[1] == "a" and parts[2] == "a":
            parts[1] = "-a"
            parts[2] = "-a"
            parts[3] = "verbe"
        elif parts[9] == "Gloss=ramasser" and parts[1] == "a" and parts[2] == "a":
            parts[1] = "-a"
            parts[2] = "-a"
            parts[3] = "verbe"
        elif parts[1]== "a" and parts[2]== "a":
            parts[3] = "préposition"
        elif parts[1]== "P.e" and parts[2]== "P.e":
            parts[1] = "e"
        elif parts[1]== "e" and parts[2]== "POSS":
            parts[3] = "POSS"
        elif parts[1]== "aat" and parts[2]== "aat":
            parts[1]= "-aat"
            parts[2]= "-aat" 
            parts[3] = "suffixe"
        elif parts[1]== "ât" and parts[2]== "ât":
            parts[1]= "-ât" 
            parts[2]= "-ât" 
            parts[3] = "suffixe"
        elif parts[1]== "at" and parts[2]== "at":
            parts[1]= "-at" 
            parts[2]= "-at" 
            parts[3] = "suffixe"
        elif parts[1]== "aad":
            parts[2]= "-aat" 
            parts[3] = "suffixe"
        elif parts[1]== "ad" and parts[2]== "ad":
            parts[1]= "-ad" 
            parts[2]= "-ad" 
            parts[3] = "suffixe"

        elif parts[2] == "kaout" and parts[3] == "verbe":
            parts[1] = "eus"
        elif parts[1] == "." and parts[2] == ".":
            parts[9] = "Gloss=."
        elif ("…" in parts[1] and len(parts[1]) > 3) or ("…" in parts[2] and len(parts[2]) > 3):
            parts[1]=parts[1].replace("…", "")
            parts[2]=parts[2].replace("…", "")
    # Traiter toujours la quatrième colonne, quelle que soit la longueur des parties
        # Règles 2-3 : Si la quatrième colonne est "adverbes" ou "pronoms", effectuer le remplacement correspondant
        elif parts[3] == "adverbes":
            parts[3] = "adverbe"
        elif parts[3] == "pronoms":
            parts[3] = "pronom"
        elif parts[3] == "complémenteurs":
            parts[3] = "complémenteur"
        elif parts[9]== "Gloss=N":
            parts[9] = "Gloss=Nom"
        elif "Gloss=" in parts[9]:
            gloss_value = parts[9].split('=',1)[1]
            new_gloss_value = gloss_value.replace('.', ' ').replace("Mut=", "|Mut=").replace("Mutchoice=", "|Mutchoice=")
            parts[9] = "Gloss=" + new_gloss_value
    return parts

def modify_column_with_wordform(parts):
    # Si la deuxième colonne et la troisième colonne sont toutes deux "P.e", effectuer un traitement spécial
    if parts[1] == "P.e" and parts[2] == "P.e":
        parts[1] = "e"  # Remplacez la deuxième colonne par "e"
    return parts

def remove_middle_spaces(word):
    """Supprimer les espaces au milieu de la chaîne, sans affecter les espaces en début et en fin de chaîne"""
    # Supprimer les espaces en début et en fin de chaîne, puis reconstituer la chaîne sans espaces au milieu
    stripped_word = word.strip()
    return word.replace(stripped_word, ''.join(stripped_word.split()))

# Début du programme principal
updated_lines = []

# Ouvrir et lire le fichier
with open('fichiers_traitement/none2_original.conllu', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Parcourir chaque ligne du fichier
for line in lines:
    stripped_line = line.strip()
    parts = stripped_line.split("\t")# Séparer le contenu de la ligne
        
    # Vérifier si la ligne commence par "# text_fr", et si c'est le cas, remplacer "'''" par un espace
    if "'''" in stripped_line:
        stripped_line = stripped_line.replace("'''", " ")
    if "# text = )" in stripped_line:
        stripped_line = stripped_line.replace("# text = )", "# text = ")
    
    parts = stripped_line.split("\t")  # Séparer le contenu de la ligne
    if stripped_line.startswith("# text = )"):
        stripped_line = stripped_line.replace("# text = )", "# text = ")  
    # Si la ligne n'est pas vide et n'est pas un commentaire, et qu'il y a suffisamment de colonnes pour effectuer les modifications nécessaires
    if parts and not stripped_line.startswith("#"):
        
        # Traitement des caractères spéciaux "-" dans la première colonne, nous avons besoin d'au moins trois colonnes pour effectuer ces opérations
        if "-" in parts[0] and len(parts) > 2:  
            parts[1] = remove_trailing_punctuation(parts[1])
            parts[1] = remove_middle_spaces(parts[1])
            parts[2] = '_'  # Supprimer le contenu de la troisième colonne
            
        # Appliquer toutes les règles de remplacement prédéfinies
        parts = apply_replacement_rules(parts)
        
    # Reconstituer la ligne et l'ajouter à la liste des résultats
    updated_line = "\t".join(parts)
    updated_lines.append(updated_line)

# Écrire les lignes mises à jour dans un nouveau fichier
with open('none2.conllu', 'w', encoding='utf-8') as output_file:
    for updated_line in updated_lines:
        output_file.write(updated_line + '\n')


# Ajouter des traits pour mots composés

### Chercher les éléments dans page pour identifier les lemmes

In [119]:
import requests
from bs4 import BeautifulSoup

def extract_words_from_url(url):
    # obtenir le contenu de la page
    response = requests.get(url)
    html_content = response.text

    # HTMl analyse et extraction des mots
    soup = BeautifulSoup(html_content, 'html.parser')
    words = []
    for word in soup.find_all('a', href=True):  
        if word.text.startswith('-'):
            modified_word = word.text
            # supprimer tous les espaces après la virgule dans tous les mots
            modified_word = modified_word.replace(', ', ',')
            words.append(modified_word)
    return words
def extract_words_from_adverbe(url):
    # obtenir le contenu de la page
    response = requests.get(url)
    html_content = response.text

    # HTMl analyse et extraction des mots
    soup = BeautifulSoup(html_content, 'html.parser')
    words = []
    for word in soup.find_all('a', href=True):  # trouver tous les liens
        if word.text.endswith('-'):
            modified_word = word.text
            # supprimer tous les espaces après la virgule dans tous les mots
            modified_word = modified_word.replace(', ', ',')
            words.append(modified_word)
    return words
# extraire les mots de la page
noms = extract_words_from_url('https://arbres.iker.cnrs.fr/index.php?title=Cat%C3%A9gorie:Noms')
verbs = extract_words_from_url('https://arbres.iker.cnrs.fr/index.php?title=Cat%C3%A9gorie:Verbes&pageuntil=Categories%0AFoeta%C3%B1#mw-pages')
adjectifs = extract_words_from_url('https://arbres.iker.cnrs.fr/index.php?title=Cat%C3%A9gorie:Adjectifs')
adverbes = extract_words_from_adverbe('https://arbres.iker.cnrs.fr/index.php?title=Cat%C3%A9gorie:Adverbes&pageuntil=Categories%0APenn-da-benn#mw-pages')

# traiter les mots avec des virgules
processed_list_noms = []
for item in noms:
    if ',' in item:
        # ajouter la chaîne d'origine
        processed_list_noms.append(item)
        # ajouter les éléments divisés
        processed_list_noms.extend(item.split(','))
    else:
        processed_list_noms.append(item)

# supprimer les espaces avant et après les éléments divisés
processed_list_noms = [element.strip() for element in processed_list_noms]

processed_list_verbs = []
for item in verbs:
    if ',' in item:
        # ajouter la chaîne d'origine
        processed_list_verbs.append(item)
        # ajouter les éléments divisés
        processed_list_verbs.extend(item.split(','))
    else:
        processed_list_verbs.append(item)

# supprimer les espaces avant et après les éléments divisés
processed_list_verbs = [element.strip() for element in processed_list_verbs]

processed_list_adjectifs = []
for item in adjectifs:
    if ',' in item:
        # ajouter la chaîne d'origine
        processed_list_adjectifs.append(item)
        # ajouter les éléments divisés
        processed_list_adjectifs.extend(item.split(','))
    else:
        processed_list_adjectifs.append(item)

# supprimer les espaces avant et après les éléments divisés
processed_list_adjectifs = [element.strip() for element in processed_list_adjectifs]

# afficher les résultats
print("Noms:",processed_list_noms)
print("Verb:", processed_list_verbs)
print("Adjectif:", processed_list_adjectifs)
print("Adverbe:", adverbes)

Noms: ["-ac'h", '-ach,-aj', '-ach', '-aj', '-achenn', '-achoù', '-ad,-iad', '-ad', '-iad', '-adeg', '-adegezh', '-adell', '-adenn', '-adez,-iadez', '-adez', '-iadez', '-adoù', '-adur', '-adurezh', '-adus', '-adusted', '-aer,-aour', '-aer', '-aour', '-af', '-ailh (N.)', '-ailhez', '-ailhoù', '-amant,-mant', '-amant', '-mant', '-an (Adj.)', '-ant,-iant', '-ant', '-iant', '-antez,-entez', '-antez', '-entez', '-aou-,-où (N.)', '-aou-', '-où (N.)', '-aoueg', '-aouennoù', '-ard,-er', '-ard', '-er', '-ardenn', '-as', '-asenn', '-atadur', '-añs,-iañs', '-añs', '-iañs', '-choù', '-ded,-ted', '-ded', '-ted', '-der,-ter', '-der', '-ter', '-ed (N.)', '-edeg', '-edigezh,-idigezh', '-edigezh', '-idigezh', '-edigoù', '-edoù', '-eg (langue)', '-eg,-og (N.)', '-eg', '-og (N.)', '-eg,-og (N.A.)', '-eg', '-og (N.A.)', '-egell,-igell,-ikell', '-egell', '-igell', '-ikell', '-egez', '-egezed', '-egezh', '-egi', '-el,-ol', '-el', '-ol', '-elezh', '-ell', '-elloù', '-en (N. coll.)', '-enenn', '-enn', '-ennad'

### Ajouter les gloses et lemmes

In [120]:
import re

with open('none2.conllu', 'r', encoding='utf-8') as file:
    lines = file.readlines()


current_entry = None
pattern = re.compile(r'^(\d+)-(\d+)\t')

for line_number, line in enumerate(lines):
    line = line.strip()
    
    if line.startswith("#"):
        continue

    match = pattern.match(line)
    if match:
        start, end = map(int, match.groups())
        gloss_values = []
        is_nom = False
        is_verb = False
        is_adjectif = False
        is_adverbe = False
        for i in range(start, end + 1):
            subsequent_line_index = line_number + (i - start + 1)
            subsequent_line = lines[subsequent_line_index].strip()
            if i == end:
                parts_subsequent_line = subsequent_line.split('\t')
                if len(parts_subsequent_line) > 1:
                    if parts_subsequent_line[1] == "a":
                        parts_subsequent_line[1] = "-a"
                        parts_subsequent_line[2] = "-a"
                        parts_subsequent_line[3] = "verb"
                        subsequent_line = '\t'.join(parts_subsequent_line)

            gloss_info = subsequent_line.split('\t')[-1]
            gloss_part = gloss_info.partition('Gloss=')[2].split('|', 1)[0]
            if gloss_part not in ["P", "p", "Nom", "sfx", "Adj","pfx","PL"]:
                gloss_values.append(gloss_part)
                ## si le mot apparait dans la liste processed_list_noms, on ajoute la valeur "nom" à la colonne 4:
            if subsequent_line.split('\t')[2] in processed_list_noms or "(PL.)" in subsequent_line.split('\t')[2] or "pluriel" in subsequent_line.split('\t')[3]:
                is_nom = True
            if subsequent_line.split('\t')[2] in processed_list_verbs:
                is_verb = True
            if subsequent_line.split('\t')[2] in processed_list_adjectifs or "(Adj.)" in subsequent_line.split('\t')[2]:
                is_adjectif = True
            if subsequent_line.split('\t')[2] in adverbes:
                is_adverbe = True
            
        if subsequent_line.split('\t')[1] in ["-oc'h", "-ig", "IMP", "superlatif","-erezh"] or subsequent_line.split('\t')[9]=="Gloss=ramasser" or subsequent_line.split('\t')[9]=="Gloss=tout-à-fait":
            gloss_values.reverse()
        original_line_parts = line.split('\t')
        original_line_parts[-1] = 'Gloss=' + ' '.join(gloss_values)
        if is_nom:
            original_line_parts[3] = 'nom'
        if is_verb:
            original_line_parts[3] = 'verbe'
        if is_adjectif:
            original_line_parts[3] = 'adjectif'
        if is_adverbe:
            original_line_parts[3] = 'adverbe'
        lines[line_number] = '\t'.join(original_line_parts) + '\n'

        
with open('none2.conllu', 'w', encoding='utf-8') as file:
    file.writelines(lines)


### Extraire tous les mots composés(optional, can pass directly)

In [8]:
import re

# Ouvrir et lire le fichier
with open('none2.conllu', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Parcourir chaque ligne du fichier
entries = []
current_entry = None
pattern = re.compile(r'^(\d+)-(\d+)\t')

# Parcourir chaque ligne du fichier
for line_index, line in enumerate(lines):
    line = line.strip()

    if line.startswith("#"):  # Commentaire, ignorer
        if current_entry is None:
            current_entry = {'header': [], 'lines': []}
        current_entry['header'].append(line)
    elif line == "":  # Ligne vide, ajouter l'entrée actuelle à la liste des entrées et réinitialiser l'entrée actuelle
        if current_entry and current_entry['lines']:
            entries.append(current_entry)
        current_entry = None
    else:
        # Vérifier si la ligne commence par un nombre suivi d'un tiret
        match = pattern.match(line)
        if match and current_entry is not None:
            start, end = map(int, match.groups())
            current_entry['lines'].append(line)  # Ajouter la ligne actuelle à l'entrée actuelle
            
            
            # Ajouter les lignes précédentes à l'entrée actuelle
            for i in range(start, end + 1):
                current_line = lines[line_index + i - (start - 1)].strip() # Récupérer la ligne correspondante
                current_entry['lines'].append(current_line)

# Ajouter la dernière entrée à la liste des entrées
if current_entry and current_entry['lines']:
    entries.append(current_entry)

# Écrire les entrées dans un nouveau fichier
with open('fichiers_traitement/mots_composes.conllu', 'w', encoding='utf-8') as output_file:
    for entry in entries:
        for header_line in entry['header']:
            output_file.write(header_line + '\n')
        for line in entry['lines']:
            output_file.write(line + '\n')
        output_file.write('\n')  # Ajouter une ligne vide entre les entrées



## Déplacer la 4ème colonne pour les UPOS

In [121]:
import re

# Ouvrir le fichier et lire son contenu
with open('none2.conllu', 'r', encoding='utf-8') as file:
    lines = file.readlines()

pattern = re.compile(r'^\d+(-\d+)?\t')  # Matcher les numéros au début de la ligne, indiquant l'ID des mots ou des tokens

# Initialiser une liste vide pour stocker les lignes modifiées
modified_lines = []

for line in lines:
    line = line.rstrip()  # Supprimer les espaces ou sauts de ligne à la fin
    # Vérifier et corriger les séquences de '\t'
    line = re.sub(r'\t\t\t', r'\t_\t_\t', line)
    line = re.sub(r'\t\t', r'\t_\t', line)
    if line.startswith("#"):  # Ajouter directement les lignes de commentaires
        modified_lines.append(line)
        continue

    match = pattern.match(line)
    if match:  # Si la ligne contient un ID de mot ou de token
        parts = line.split('\t')
        if len(parts) > 4:  # Assurer que la ligne contient suffisamment de parties pour être traitée
            # Déplacer le contenu du 5ème champ au 6ème champ, si le 6ème champ n'existe pas, l'ajouter
            if len(parts) > 5:
                parts[5] = parts[4]  # Si un 6ème champ existe déjà, remplacer son contenu par celui du 5ème champ
            else:
                parts.append(parts[4])  # Si aucun 6ème champ, ajouter le contenu du 5ème champ

            # Déplacer le contenu du 4ème champ au 5ème et mettre "_" dans le 4ème champ
            parts[4] = parts[3]
            parts[3] = "_"
            
            modified_line = '\t'.join(parts)  # Recombiner la ligne
            modified_lines.append(modified_line)
        else:
            # Si la ligne ne contient pas assez de parties, l'ajouter telle quelle
            modified_lines.append(line)
    else:
        # Pour les lignes qui ne commencent pas par un numéro, les ajouter directement
        modified_lines.append(line)

# Écrire le contenu modifié dans le fichier
with open('none2.conllu', 'w', encoding='utf-8') as file:
    for line in modified_lines:
        file.write(line + '\n')  # Écrire les lignes modifiées, en ajoutant un saut de ligne à la fin



# Séparer les exemples dans none2.conllu selon les dialects

In [122]:
# List of your dialect categories
dialect_categories = ['léonard', 'cornouaillais', 'trégorrois', 'vannetais', 'breton central', 'standard']

# Adding 'inconnu' category for unmatched examples
dialect_categories.append('inconnu')

# Dictionary to hold the examples for each dialect
examples_by_dialect = {dialect: [] for dialect in dialect_categories}

# Function to check if the dialect line contains any of the specified dialects
def contains_dialect(line, dialects):
    for dialect in dialects:
        if dialect.lower() in line.lower():
            return dialect
    return 'inconnu'  # Return 'inconnu' if no match is found

# Read the file and collect examples
with open('none2.conllu', 'r', encoding='utf-8') as file:
    examples = file.read().strip().split('\n\n')

# Process each example
for example in examples:
    lines = example.split('\n')
    dialect_line = next((line for line in lines if line.startswith('# dialect =')), None)
    matched_dialect = contains_dialect(dialect_line, dialect_categories[:-1]) if dialect_line else 'inconnu'
    examples_by_dialect[matched_dialect].append(example + '\n')

# Create the directory if it doesn't exist
import os
output_directory = "bretonconlls"
os.makedirs(output_directory, exist_ok=True)

# Save examples into separate files based on dialect and update the sent_id
for dialect, examples in examples_by_dialect.items():
    filename = os.path.join(output_directory, f"{dialect.lower().replace(' ', '_')}.conllu")
    with open(filename, 'w', encoding='utf-8') as output_file:
        for index, example in enumerate(examples, 1):
            lines = example.split('\n')
            for i, line in enumerate(lines):
                if line.startswith("# sent_id ="):
                    parts = line.split("__")
                    lines[i] = f"{parts[0]}__{index}"
            updated_example = '\n'.join(lines)
            output_file.write(updated_example + '\n')  # Double newline to separate examples in the file
